This is a Ngrams and Word2Vec training and testing procedure.
The notebook supports various model options and has support for less common variants, like prediction of only the POS.

This notebook requires Google Drive to be used as temporary storage.
This notebook has been **deprecated** as there are more userfriendly
**ngram.ipynb** and **word2vec.ipynb** notebooks. This notebook has been included for reference, because some of ideas tested in course project, are not part of these cleaned up notebooks.


# Setup

In [ ]:
# @title Mount Google Drive to get started. Click `Show code` to view details. { display-mode: "form" }
# @markdown The Notebook will cache generated models and other data on Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Navigate to folder containing AES files

In [ ]:
# @title Navigate on Google Drive to `/content/drive/MyDrive/Colab` `Notebooks/data/`. Click `Show code` to view details. { display-mode: "form" }

# Two possible spellings for directory path
%cd /content/drive
try:
  %cd /content/drive/MyDrive/Colab Notebooks/data/
except:
  %cd /content/drive/My Drive/Colab Notebooks/data/
# To Print directory and its contents, uncomment next lines
# %pwd
# %ls

/content/drive
/content/drive/MyDrive/Colab Notebooks/data


In [ ]:
# @title Check files exist on Google Drive. Click `Show code` to view details. { display-mode: "form" }
import os

aes_filenames = [
    "_aes_bbawamarna.json", "_aes_bbawhistbiospzt.json",
    "_aes_sawmedizin.json", "_aes_bbawarchive.json",
    "_aes_bbawpyramidentexte.json", "_aes_bbawbriefe.json",
    "_aes_bbawramessiden.json", "_aes_smaek.json",
    "_aes_bbawfelsinschriften.json", "_aes_bbawtempelbib.json", "_aes_tb.json",
    "_aes_bbawgrabinschriften.json", "_aes_bbawtotenlit.json",
    "_aes_tuebingerstelen.json", "_aes_bbawgraeberspzt.json",
    "_aes_sawlit.json" ]
gaps_file = "all_gaps_id.txt"
aed_dictionary = "dictionary.json"
ge_en_dict = "ge_en_dictionary.json"

for fi in aes_filenames + [aed_dictionary, ge_en_dict, gaps_file]:
  if os.path.exists(fi) != True:
    print(f"file {fi} does not exist on Google Drive")
    raise Exception("Missing file(s)")

# Make sure ramses-trl directory exists.
for di in [ 'ramses-trl', 'ramses-trl/data' ]:
  if os.path.exists(di) != True:
    os.mkdir(di)

if os.path.exists('ramses-trl/data/src-train.txt') != True:
  rt_url = 'https://zenodo.org/records/4954597/files/ramses-trl_2021_05_29.zip?download=1'
  import requests
  print(f"Loading {rt_url}")
  response = requests.get(rt_url)
  open("ramses-trl_2021_05_29.zip", "wb").write(response.content)
  !unzip ramses-trl_2021_05_29.zip

rtc_path = 'ramses-trl/data/'
rtc_ext = '.txt'
rtc_src_files = [ 'src-train', 'src-val' ] # Use -train for training/gaps and -val for testing/gaps.
rtc_tgt_files = [ 'tgt-train', 'tgt-val' ]
rtc_files = [rtc_path+fname+rtc_ext for fname in rtc_src_files + rtc_tgt_files]
all_files = aes_filenames + [aed_dictionary] + rtc_files
for fi in all_files:
  if os.path.exists(fi) != True:
    print(f"file {fi} does not exist on Google Drive")
    raise Exception("Missing file(s)")
print("\U0001f44D All required files located.")

👍 All required files located.


In [ ]:
# @title Install required libraries. { display-mode: "form" }
#%pip install pympler
%pip install Levenshtein
%pip install unidecode
%pip install geneticalgorithm2
%pip install func_timeout
%pip install gardiner2unicode
%pip install wikitextparser

In [ ]:
# @title Load required libraries. { display-mode: "form" }
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import json
import io
import re
import string
import random
from tqdm.notebook import tqdm, tqdm_notebook
import numpy as np
import collections
import gensim.models
import Levenshtein
import heapq
from unidecode import unidecode
from geneticalgorithm2 import geneticalgorithm2 as ga2
from collections import Counter
import time
from tabulate import tabulate
from gardiner2unicode import GardinerToUnicodeMap
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
from gensim.models.callbacks import CallbackAny2Vec

Import libraries (all libraries should be installed above)

# Data extraction

In [ ]:
# @title Parse AES Files from Google Drive into `aes_file_contents`, `aes_sentence_dictionary`. Click `Show code` to view details. { display-mode: "form" }

aes_print_sentences = []
written_form_to_lemma_id = dict()
written_form_to_mdc = dict()
def read_aes_files():
  aes_file_contents = []
  aes_sentence_dictionary = {}
  global aes_print_sentences

  notebook = tqdm_notebook(aes_filenames,desc="Loading...")
  for filename in notebook:
    # go through each file
    notebook.desc = filename
    with open(filename, "r") as file:
      data = json.load(file)
      sentences = []

      # go through all elements (sentences) in the JSON data
      for e in data:
        sentence = []
        aes_sentence_dictionary[e] = {}

        sentence_object = data[e]
        token_list = sentence_object["token"]

        for token in token_list:
          # written_form and mdc exist for all tokens
          cotext_translation = token["cotext_translation"] if "cotext_translation" in token else ""
          lemma_form = token["lemma_form"] if "lemma_form" in token else ""
          lemma_id = token["lemmaID"] if "lemmaID" in token else "-1"
          pos = token["pos"] if "pos" in token else ""
          token_object = (token["mdc"], cotext_translation, lemma_form, lemma_id, pos)
          if 'written_form' in token:
            written_form_to_lemma_id[token['written_form']] = lemma_id
          if 'written_form' in token:
            written_form_to_mdc[token['written_form']] = token["mdc"]
          sentence.append(token_object)

        sentences.append(sentence)

        aes_sentence_dictionary[e]["text"] = sentence_object["text"]
        aes_sentence_dictionary[e]["translation"] = sentence_object["sentence_translation"]
        aes_sentence_dictionary[e]["corpus"] = sentence_object["corpus"]
        aes_sentence_dictionary[e]["date"] = sentence_object["date"]
        aes_sentence_dictionary[e]["tokens"] = sentence
        aes_sentence_dictionary[e]["token_list"] = "<s> " + " ".join([token[3] for token in sentence]) + " </s>" # list of token lemma IDs separated by whitespace
        aes_sentence_dictionary[e]["sentence_transliteration"] = " ".join([token[0] for token in sentence])
        # You may add here search term for interesting sentences. I search for sentence with numbers and fractions.
        if len(aes_print_sentences) < 20: # First 20
            aes_print_sentences.append(aes_sentence_dictionary[e])
        if len(aes_print_sentences) < 100 and sum(["5" in token["mdc"] for token in token_list]) and sum(["1/8" in token["mdc"] for token in token_list]):
            aes_print_sentences.append(aes_sentence_dictionary[e])

      aes_file_contents.append(sentences)
  notebook.desc = "Loading AES"
  return (aes_file_contents, aes_sentence_dictionary)
aes_file_contents, aes_sentence_dictionary = read_aes_files()
print(f"AES Loaded: {len(aes_file_contents)} files with {len(aes_sentence_dictionary.items())} sentences")

# Show interesting sentences. The reason to use subset is that all
# sentences could be too much.
aes_print_sentences_compact = [[sentence['text'], sentence['token_list'], sentence['sentence_transliteration']] for sentence in aes_print_sentences[:100]]
pd.set_option('display.max_rows', 5)
aes_print_dataframe = pd.DataFrame(aes_print_sentences_compact,
                         columns=['Text', 'Token list', 'Transliteration'])
aes_print_dataframe


Loading...:   0%|          | 0/16 [00:00<?, ?it/s]

AES Loaded: 16 files with 101796 sentences


,Text,Token list,Transliteration
0,25IKMZZWRBARVG5WSM7JVGVBDE,<s> -1 856314 -1 </s>,"[...] [nfr-xpr,pl]-raw-wa-[n-raw] [...]"
1,25IKMZZWRBARVG5WSM7JVGVBDE,<s> -1 73330 850814 64830 550077 851446 51990 ...,"[...] [mH] 4 [m-mj,tt] ⸢SAa⸣-m pA [wD] [rs,j-j..."
...,...,...,...
22,S36FKQSICNFDLADBECABA64XXE,<s> 26870 10100 69410 110440 850815 850815 850...,"jni.n =sn 〈m〉fkA,t HqA,t 14 1/4 1/8 1/32 (w)DA..."
23,S36FKQSICNFDLADBECABA64XXE,<s> 26870 10100 112330 178610 850814 850814 -1...,jni.n =sn HD dbn 1676 1/2 [...] ⸢Hzmn⸣ ⸢dbn⸣ ⸢...


In [ ]:
# @title Create test set of AES sentences. Click `Show code` to view details. { display-mode: "form" }

# Divide randomically 20% of sentences to test.
def is_test(token_list):
  h = hash(token_list)
  return True if (h % 100) < 20 else False # Select 20% inputs

def get_aes(ae_dictionary):
  aecorpus = collections.namedtuple('AncientEgyptianCorpus', 'aes aes_1 aes_t aes_g aesen aesen_1 aesen_t aed rtc rtc_1 rtc_t rtc_g')

  aes_g = []
  aes = []
  aes_1 = []
  aes_t = []
  c = 5
  print("Examples of sentences with gaps:")
  for id, sentence_all in ae_dictionary.items():
    tokens = sentence_all["tokens"]
    gaps = True
    try:
      # Consider unreadable as gap.
      gaps = any(token[3] == "-1" for token in tokens)
      if gaps == True and c > 0:
        c -= 1
        print(sentence_all["token_list"], "    ", sentence_all["sentence_transliteration"])
    except:
      #print("Gaps check fail: ", sentence)
      pass
    if gaps:
      aes_g.append(sentence_all)
    else:
      cl = is_test(sentence_all["token_list"]) # classification is per token sequence.
      aes.append(sentence_all)
      if cl == False:
        aes_1.append(sentence_all)
      else:
        aes_t.append(sentence_all)

  aecorpus.aes_g = aes_g
  aecorpus.aes = aes
  aecorpus.aes_t = aes_t
  aecorpus.aes_1 = aes_1
  return aecorpus

aecorpus = get_aes(aes_sentence_dictionary)
print(f"AES partitioned: usable sentences={len(aecorpus.aes)} incl: training={len(aecorpus.aes_1)} test={len(aecorpus.aes_t)} gapped: {len(aecorpus.aes_g)}")

Examples of sentences with gaps:
<s> -1 856314 -1 </s>      [...] [nfr-xpr,pl]-raw-wa-[n-raw] [...]
<s> -1 73330 850814 64830 550077 851446 51990 96090 -1 </s>      [...] [mH] 4 [m-mj,tt] ⸢SAa⸣-m pA [wD] [rs,j-jmn,tj] [...]
<s> -1 851809 78870 33360 850814 -1 </s>      [...] jri.w [n] ⸢jtr,w⸣ [6] [...]
<s> 119600 851427 854537 851446 850814 51990 550077 851446 182830 20570 -1 </s>      [xr] [jr] ⸢Xnw⸣ pA 4 wD ⸢SAa-m⸣ pA [Dw] [jAb,tj] [...]
<s> 21881 10090 78870 32820 10030 859387 856317 -1 64360 182830 64360 114300 64360 141480 64360 66830 -1 64360 158000 64360 30750 81660 851809 851446 850317 550021 10030 32820 142050 10100 91900 86570 181400 </s>      [jw] =[s] [n] [jt(j)] =[j] [raw-Hr,w-Ax,tj-Hai-m-Ax,t] [m-rn≡f-m-Sw-n,tj-m-jtn] [...] [m] ⸢Dw.pl⸣ [m] ⸢xAs,t.pl⸣ m sx,t.pl m mAw,t.pl [...] m StA.pl [m] [(j)x,t] [nb.t] jri.w pA jtn ⸢pAy⸣ =j jt(j) ⸢sxpr⸣ =sn r nHH [D,t]
AES partitioned: usable sentences=79357 incl: training=63938 test=15419 gapped: 22439


In [ ]:
from transformers import BertTokenizer, pipeline, AutoModelForMaskedLM
%pushd snlp-project/tokenizers/m-bert
egy_mbert_tokenizer = BertTokenizer.from_pretrained('m-bert-combined-tokenizer')
%popd

/content/drive/MyDrive/Colab Notebooks/data/snlp-project/tokenizers/m-bert


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


/content/drive/MyDrive/Colab Notebooks/data
popd -> /content/drive/MyDrive/Colab Notebooks/data


In [ ]:
sentences = []
dev_sentences = []
eval_sentences = []

dev_file_name = "snlp-project/preprocessing/final_files/intact/dev/all_dev.txt"
test_file_name = "snlp-project/preprocessing/final_files/intact/test/all_test.txt"

print("Opening file:", dev_file_name)
dev_file = open(dev_file_name, "r")
dev_file_lines = dev_file.readlines()

for line in dev_file_lines:
  sentences.append(line.replace("\n", ""))
  dev_sentences.append(line.replace("\n", ""))

dev_file.close()

print("Opening file:", test_file_name)
test_file = open(test_file_name, "r")
test_file_lines = test_file.readlines()

for line in test_file_lines:
  sentences.append(line.replace("\n", ""))
  eval_sentences.append(line.replace("\n", ""))

test_file.close()

Opening file: snlp-project/preprocessing/final_files/intact/dev/all_dev.txt
Opening file: snlp-project/preprocessing/final_files/intact/test/all_test.txt


In [ ]:
print(sentences[:10])
print(dev_sentences[:10])
print(eval_sentences[:10])

['jꜣ,w n kꜣ =k n stw,t =k', 'bn ftt =tw =f', 'jri̯ =tw n =k ḥtp-ḏi̯-nzw m tʾ =k ḥnq,t n pr =k', 'hrw pn swn.t jri̯.n mnj,w Mss-jꜣ m-ꜥ ꜥnḫ,t-n,t-nʾ,t Pꜣ-jḥ ḥnꜥ zꜣ =s wꜥb Mnw hrw 17 m ḥm,t Ḫr,jt ḥnꜥ hrw 4 m ḥm,t Ḥnw,t', 'ꜥnḫ jtj =j Rꜥw-ḥqꜣ-ꜣḫ,tj-ḥꜥi̯-m-ꜣḫ,t M-rn=f-m-Rꜥw-jtj-jy-m-Jtn ḏi̯ ꜥnḫ ḏ,t nḥḥ', 'ḏi̯ =f tʾ ḥnq,t qbḥ,w ḥtp m Jwn,w n kꜣ n ḥr,j-jr,w-nbw-pꜣq-n-pr-Jtn Ptḥ-my mꜣꜥ-ḫrw', 'rʾ =j ẖr mꜣꜥ,t', 'dqr ꜥ 12', 'jm jri̯.y =f pꜣy =k ꜥḥꜥ,w', 'Mꜣꜥ,t zꜣ,t-Rꜥw']
['jꜣ,w n kꜣ =k n stw,t =k', 'bn ftt =tw =f', 'jri̯ =tw n =k ḥtp-ḏi̯-nzw m tʾ =k ḥnq,t n pr =k', 'hrw pn swn.t jri̯.n mnj,w Mss-jꜣ m-ꜥ ꜥnḫ,t-n,t-nʾ,t Pꜣ-jḥ ḥnꜥ zꜣ =s wꜥb Mnw hrw 17 m ḥm,t Ḫr,jt ḥnꜥ hrw 4 m ḥm,t Ḥnw,t', 'ꜥnḫ jtj =j Rꜥw-ḥqꜣ-ꜣḫ,tj-ḥꜥi̯-m-ꜣḫ,t M-rn=f-m-Rꜥw-jtj-jy-m-Jtn ḏi̯ ꜥnḫ ḏ,t nḥḥ', 'ḏi̯ =f tʾ ḥnq,t qbḥ,w ḥtp m Jwn,w n kꜣ n ḥr,j-jr,w-nbw-pꜣq-n-pr-Jtn Ptḥ-my mꜣꜥ-ḫrw', 'rʾ =j ẖr mꜣꜥ,t', 'dqr ꜥ 12', 'jm jri̯.y =f pꜣy =k ꜥḥꜥ,w', 'Mꜣꜥ,t zꜣ,t-Rꜥw']
['Rꜥw-Ḥr,w-ꜣḫ,tj-ḥꜥi̯-m-ꜣḫ,t M-rn=f-m-šw-n,tj-m-Jtn ḏi̯ ꜥnḫ ḏ,t nḥḥ Jtn ꜥn

In [ ]:
# This is how masked sentences are constructed (mainly for reference)
test_sentences = []
test_file_name = "snlp-project/preprocessing/final_files/intact/test/all_test.txt"

print("Opening file:", test_file_name)
test_file = open(test_file_name, "r")
test_file_lines = test_file.readlines()

random.seed(0)

for line in test_file_lines:
  sentence = line.replace("\n", "")
  masked_tokens = []
  tokenized_sentence = egy_mbert_tokenizer.tokenize(sentence)
  mt_index = random.randrange(0, len(tokenized_sentence))
  masked_token = tokenized_sentence[mt_index]
  tokenized_sentence[mt_index] = "[MASK]"
  test_sentences.append((sentence, " ".join(tokenized_sentence), masked_token))

test_file.close()

Opening file: snlp-project/preprocessing/final_files/intact/test/all_test.txt


In [ ]:
# Read masked sentences to use same mask indexes.
test_masked_file_name = "snlp-project/predictions/all_test_masked.txt"
test_sentences = []

print("Opening file:", test_masked_file_name)
test_masked_file = open(test_masked_file_name, "r")
test_masked_file_lines = test_masked_file.readlines()

for sentence in test_masked_file_lines:
  line = sentence.replace(u'\xa0', u' ') # account for non-breaking spaces?
  line = line.replace("\n", "")
  original_sentence, masked_sentence, correct_prediction = line.split(" | ")
  test_sentences.append((original_sentence, masked_sentence, correct_prediction))

test_masked_file.close()
for sentence in test_sentences[:5]:
    print(test_sentences[:5])

Opening file: snlp-project/predictions/all_test_masked.txt
[('Raw-Hr.w-Ax.ty-Hai-m-Ax.t M-rn=f-m-Sw-n.ty-m-Jtn Di anx D.t nHH Jtn anx wr ym.y-HAb-sd nb-Snn.t-nb-Jtn nb-p.t nb-tA m Pr-Jtn m Ax.t-Jtn', 'Raw-Hr.w-Ax.ty-Hai-m-Ax.t M-rn=f-m-Sw-n.ty-m-Jtn Di anx D.t nHH Jtn anx wr ym.y-HAb-sd nb-Snn.t-nb-Jtn nb-p.t [MASK] m Pr-Jtn m Ax.t-Jtn', 'nb-tA'), ('Hw.t-Hr.w nb.t-Hw.t', 'Hw.t-Hr.w [MASK]', 'nb.t-Hw.t'), ('aA =y m tm m ra =y wDA nHi.n =f yAw nfr mri anx', '[MASK] =y m tm m ra =y wDA nHi.n =f yAw nfr mri anx', 'aA'), ('xr yr Xnw pA 4 wD SAa-m pA Dw yAb.ty n Ax.t-Jtn nfr.yt-r pA Dw ymn.ty n Ax.t-Jtn Ax.t-Jtn r-Ha.w =s tAy', 'xr yr Xnw pA 4 wD SAa-m pA [MASK] yAb.ty n Ax.t-Jtn nfr.yt-r pA Dw ymn.ty n Ax.t-Jtn Ax.t-Jtn r-Ha.w =s tAy', 'Dw'), ('bn yri.y HyHy =f', 'bn yri.y HyHy [MASK]', '=f')]
[('Raw-Hr.w-Ax.ty-Hai-m-Ax.t M-rn=f-m-Sw-n.ty-m-Jtn Di anx D.t nHH Jtn anx wr ym.y-HAb-sd nb-Snn.t-nb-Jtn nb-p.t nb-tA m Pr-Jtn m Ax.t-Jtn', 'Raw-Hr.w-Ax.ty-Hai-m-Ax.t M-rn=f-m-Sw-n.ty-m-Jtn Di anx D.

In [ ]:
train_sentences = []

train_file_name = "snlp-project/preprocessing/final_files/intact/dev/all_train.txt"

print("Opening file:", train_file_name)
train_file = open(train_file_name, "r")
train_file_lines = train_file.readlines()

for line in train_file_lines:
  train_sentences.append(line.replace("\n", ""))

train_file.close()

print("Opening file:", train_file_name)
train_file = open(train_file_name, "r")
train_file_lines = train_file.readlines()

for line in train_file_lines:
  sentences.append(line.replace("\n", ""))
  eval_sentences.append(line.replace("\n", ""))

train_file.close()

Opening file: snlp-project/preprocessing/final_files/intact/dev/all_train.txt
Opening file: snlp-project/preprocessing/final_files/intact/dev/all_train.txt


In [ ]:
# @title Read Ancient Egyptian Dictionary. Click `Show code` to view details. { display-mode: "form" }

def read_aed():
  aed_dict = {}

  with open(aed_dictionary, "r") as dictionary_file:
    dictionary_data = json.load(dictionary_file)
  dictionary_entries = dictionary_data["TEI"]["text"]["body"]["entry"]

  for entry in dictionary_entries:
    xml_id = entry["_xml:id"]
    lemma_id = xml_id[3:]
    entry_translations = [(e["quote"], e["_xml:lang"]) for e in entry["sense"]["cit"] if "quote" and "_xml:lang" and "_type" in e and e["_type"] == "translation"]
    aed_dict[lemma_id] = {
        "form": entry["form"]["orth"],
        "grammarGroup": entry["gramGrp"]["term"],
        "translations": entry_translations
    }

  print('Example entry in the dictionary:')
  print(aed_dict["10"])

  print('The size of the dictionary is:', len(aed_dict))

  unique_lemma_ids = set([token[3] for file in aes_file_contents for sentence in file for token in sentence])
  missing_vocabulary = [lemma_id for lemma_id in unique_lemma_ids if lemma_id not in aed_dict.keys()]
  missing_vocabulary_count = len(missing_vocabulary) - 1 # account for lemma_id -1 for lacunae
  print('The number of missing vocabulary is:', missing_vocabulary_count)

  vocabulary_items_with_en_translations = {k: v for k, v in aed_dict.items() if any(translation[1] == 'en' for translation in v["translations"])}
  print('The size of the English-only dictionary:', len(vocabulary_items_with_en_translations))
  print('Portion of vocabulary in the dictionary with English translations:', len(vocabulary_items_with_en_translations)/len(aed_dict))
  return aed_dict
aecorpus.aed = read_aed()

Example entry in the dictionary:
{'form': 'ꜣj.wj', 'grammarGroup': 'substantive/substantive_masc', 'translations': [('Doppelverband, Kreuzverband (?) (med.)', 'de'), ('pair of bandages (med.)', 'en')]}
The size of the dictionary is: 35052
The number of missing vocabulary is: 0
The size of the English-only dictionary: 16971
Portion of vocabulary in the dictionary with English translations: 0.484166381376241


Create English translations where applicable and attach them to AES corpus.

In [ ]:
# @title Add English Translations to Ancient Egyptian Sentences. Click `Show code` to view details. { display-mode: "form" }
from textblob import TextBlob
from textblob.exceptions import NotTranslated, TranslatorError
import json
ge_en = dict() # A small dictionary for mappings texts to English
ge_en_changed = False
ge_en_new_entries = 0

if os.path.exists("ge_en_dictionary.json"):
  with open('ge_en_dictionary.json') as f:
    ge_en = json.load(f)

def new_translation_from_de(de):
  blob = TextBlob(de)
  try:
    res = blob.translate(from_lang='de', to='en')
  except NotTranslated:
    res = de
  except TranslatorError:
    res=" "

  global ge_en
  ge_en[de] = str(res)
  global ge_en_changed
  ge_en_changed = True
  global ge_en_new_entries
  ge_en_new_entries += 1
  # If added over 2000 entries it is good to store the database.
  if ge_en_new_entries >= 2000:
    with open('ge_en_dictionary.json', 'w') as f:
      json.dump(ge_en, f)
      print(f"Stored updated ge/en dictionary: {len(ge_en)} entries")
    ge_en_new_entries = 0
    ge_en_changed = False
  return str(res)

def add_english_translations(aes_dictionary, aed_dictionary):
  translations = 0
  translations_full = 0
  translations_full_lookup = 0
  translations_part = 0
  translations_part_lookup = 0
  en_translations = dict()
  de_translations = dict()
  for id,ent in aed_dictionary.items():
    t = list(filter(lambda translation: translation[1] == 'en', ent["translations"]))
    if len(t) > 0:
      en_translations[id] = str(t[0][0])
      translations += 1
    t_de = list(filter(lambda translation: translation[1] == 'de', ent["translations"]))
    if len(t) > 0:
      de_translations[id] = str(t_de[0][0])
  print(f"Using dictionary with {len(en_translations.items())} translations")
  translations = 0
  for id, sentence in tqdm_notebook(aes_sentence_dictionary.items(),desc="Translating"):
    if 'translation' in sentence and len(sentence['translation']) > 1:
      # Found german translation, using it.
      de_translations = sentence['translation']
      if de_translations in ge_en:
        sentence["en"] = ge_en[de_translations]
        if translations_full_lookup < 5:
          print('FULL', sentence["text"], sentence["sentence_transliteration"], sentence["en"])
        translations_full_lookup +=1
      else:
        res = new_translation_from_de(de_translations)
        sentence["en"] = res
        if translations_full < 5:
          print('FULL', sentence["text"], sentence["sentence_transliteration"], sentence["en"])
        translations_full += 1
      continue # Translated full sentence
    for token in sentence["tokens"]:
      lemma_id = token[3]
      if lemma_id not in en_translations:
        if lemma_id in de_translations:
          de_translation = de_translations[lemma_id]
          if de_translation in ge_en:
            en_translations[lemma_id] = ge_en[de_translation]
            if translations_part_lookup < 5:
              print('PART', sentence["text"], sentence["sentence_transliteration"], "PART:", en_translations[lemma_id])
            translations_part_lookup += 1
            continue
          else:
            res = new_translation_from_de(de_translation)
            en_translations[lemma_id] = res
            if translations_part < 5:
              translations_part += 1
              print('PART', sentence["text"], sentence["sentence_transliteration"], "PART:", en_translations[lemma_id])
        break
    else:
      translated_tokens = [en_translations[token[3]] for token in sentence["tokens"]]
      sentence["en"] = "  ".join(translated_tokens)
      if translations < 5:
        print('CONSTRUCTED', sentence["text"], sentence["sentence_transliteration"], sentence["en"])
      translations += 1
  print(f"AES: Used {translations_full + translations_full_lookup} English sentence translation from de")
  print(f"AES: Constructed {translations_part + translations_part_lookup} English sentence part translations from de")
  print(f"AES: Constructed {translations} English translations from dictionary")

add_english_translations(aes_sentence_dictionary, aecorpus.aed)

aecorpus.aesen = list([sentence for sentence in aecorpus.aes if "en" in sentence and "translation" in sentence])
aecorpus.aesen_1 = list([sentence for sentence in aecorpus.aes_1 if "en" in sentence and "translation" in sentence])
aecorpus.aesen_t = list([sentence for sentence in aecorpus.aes_t if "en" in sentence and "translation" in sentence])
aecorpus_readable = list([[sentence['text'], sentence['sentence_transliteration'], sentence['en'], sentence['translation']] for sentence in aecorpus.aesen])

pd.DataFrame(aecorpus_readable, columns=['sentence id', 'transliteration', 'translation', 'de translation'])

# Store updated ge/en dictionary if it has been updated
if ge_en_new_entries >= 5:
  #print(list(ge_en.items())[:10])
  with open('ge_en_dictionary.json', 'w') as f:
    json.dump(ge_en, f)
    print(f"Updated ge/en dictionary: {len(ge_en)} entries")

Using dictionary with 16971 translations


Translating:   0%|          | 0/101796 [00:00<?, ?it/s]

FULL 25IKMZZWRBARVG5WSM7JVGVBDE [...] [nfr-xpr,pl]-raw-wa-[n-raw] [...] ... [Nefer-cheper] -re-Wa [en-re] ...
FULL 25IKMZZWRBARVG5WSM7JVGVBDE [...] [mH] 4 [m-mj,tt] ⸢SAa⸣-m pA [wD] [rs,j-jmn,tj] [...] ... 4 Ellen, [equally] from the [southwestern stele] ...
FULL 25IKMZZWRBARVG5WSM7JVGVBDE [...] jri.w [n] ⸢jtr,w⸣ [6] [...] ... it makes [6] JTr, W lengths ...
FULL 25IKMZZWRBARVG5WSM7JVGVBDE [xr] [jr] ⸢Xnw⸣ pA 4 wD ⸢SAa-m⸣ pA [Dw] [jAb,tj] [...] [As for the] area within these four steles [concerns], starting with the [eastern mountain] ...
FULL 25IKMZZWRBARVG5WSM7JVGVBDE [jw] =[s] [n] [jt(j)] =[j] [raw-Hr,w-Ax,tj-Hai-m-Ax,t] [m-rn≡f-m-Sw-n,tj-m-jtn] [...] [m] ⸢Dw.pl⸣ [m] ⸢xAs,t.pl⸣ m sx,t.pl m mAw,t.pl [...] m StA.pl [m] [(j)x,t] [nb.t] jri.w pA jtn ⸢pAy⸣ =j jt(j) ⸢sxpr⸣ =sn r nHH [D,t] [And it belongs to my father] "[Re-Harachten, who cheers sunlight in his name, which is in the sun]" (= aton), ... [consisting of] mountains and deserts, from Weideland, new territory and ... from tree pla

In [ ]:
# @title Lemmatization for RTC texts (partially incomplete). Click `Show code` to view details. { display-mode: "form" }
# Incomplete conversion from text to lemma form
# Unable to handle cases where there are multiple translations etc.

verbose_lemmatize=False # @param {type:"boolean"}

repl_dict = { 'A': 'ꜣ', 'H': 'ḥ', 'S': 'š', ',': '.', 'D': 'ḏ', 'x': 'ḫ', 'a': 'ꜥ', 's': 'z', 'i': 'i̯' }
rem_list = [ '⸢', '⸣', '[', ']', '(', ')', '{', '}', '<', '>', '⸮', '?' ]
lemmatize_dict = dict()
lemmatize_dict_id = dict()
lemma_id_dict = dict()
def lemmatize(txt):
  global repl_dict
  global rem_list
  global lemmatize_dict
  if txt in lemmatize_dict:
    return lemmatize_dict[txt]
  remtxt = ''
  for char in txt: # Remove '⸢', '⸣', '[', ']' etc.
    if char not in rem_list:
      remtxt += char
  if remtxt in lemmatize_dict:
    return lemmatize_dict[remtxt]
  if remtxt.isnumeric():
    return '1...n'
  out = []
  for char in remtxt:
    if char in repl_dict:
      out.append(repl_dict[char])
    else:
      repl_dict[char] = char
      out.append(char)
  out_str = "".join(out)
  # Some replacements I've seen. Not complete list.
  out_str = out_str.replace('jtn', 'Jtn')
  out_str = out_str.replace('ꜣḫ', 'Ꜣḫ')
  out_str = out_str.replace('.tt', '.tjt')
  lemmatize_dict[txt] = out_str
  return out_str

def get_lemma_id(lemma):
  global lemma_id_dict
  if lemma in lemma_id_dict:
     return lemma_id_dict[lemma]
  remtxt = ''
  for char in lemma: # Remove '⸢', '⸣', '[', ']' etc.
    if char not in rem_list:
      remtxt += char
  if remtxt in lemma_id_dict:
     return lemma_id_dict[remtxt]
  return None

mismatch_c = 0

def add_to_lemmatize_dict(txt, lemma, lemmaid, id, transl, transl2):
  global mismatch_c
  global lemmaize_dict
  if txt in lemmatize_dict:
    if lemma != lemmatize_dict[txt] and mismatch_c < 10 and verbose_lemmatize:
      print('Lemmatization mismatch: ', id, "v", lemmatize_dict_id[txt], txt, lemma, lemmatize_dict[txt])
      print(transl)
      print(transl2)
      mismatch_c += 1
    return
  lemmatize_dict[txt] = lemma
  lemma_id_dict[txt] = lemmaid
  lemmatize_dict_id[txt] = id
  remtxt = ''
  for char in txt:
    if char not in rem_list:
      remtxt += char
  if remtxt in lemmatize_dict:
    return
  lemmatize_dict[remtxt] = lemma
  lemma_id_dict[remtxt] = lemmaid
  lemmatize_dict_id[remtxt] = id

print("Lemmatization db")
add_to_lemmatize_dict('LACUNA', 'LACUNA', -1, 'LACUNA', 'LACUNA', 'LACUNA')
add_to_lemmatize_dict('1...n', '1...n', "850814", '1...n', '1...n', '1...n')

# Processing numbers
def add_numbers_to_lemmatize():
  digits = list([digit for digit in range(1,10)])
  magnitudes = list([pow(10,magnitude) for magnitude in range(7)])
  for num in [digit * magnitude for magnitude in magnitudes for digit in digits]:
    add_to_lemmatize_dict(str(num), '1...n', "850814", '1...n', '1...n', '1...n')
  for sfraction in range(1,9):
    add_to_lemmatize_dict("1/" + str(pow(2, sfraction)), '1...n', "850814", '1...n', '1...n', '1...n')
  for fraction in range(3,9):
    add_to_lemmatize_dict(f"1/{fraction}", '1...n', "850814", '1...n', '1...n', '1...n')
  add_to_lemmatize_dict("2/3", '1...n', "850814", '1...n', '1...n', '1...n')
  add_to_lemmatize_dict("3/4", '1...n', "850814", '1...n', '1...n', '1...n')
  add_to_lemmatize_dict("3/8", '1...n', "850814", '1...n', '1...n', '1...n')
  for fraction in range(1,10):
    add_to_lemmatize_dict("1/" + str(10*fraction), '1...n', "850814", '1...n', '1...n', '1...n')

add_numbers_to_lemmatize()

tr = set()
for sentence in aecorpus.aes:
  for token in sentence['tokens']:
    triplet = (token[3], token[0], token[2])
    tr.add(triplet)
    add_to_lemmatize_dict(triplet[1], triplet[2], triplet[0], sentence['text'], sentence['sentence_transliteration'], [token[2] for token in sentence['tokens']])

print("Built lemmatization database")

Lemmatization db
Built lemmatization database


There are differences between RTC and AES/AED transliterations. In order to investigate the differances, this short function `lemma_id_closest` was written. The function is generally not fast enough to run for all the words, but may prove useful when debugging the differences.

In [ ]:
# @title Debug tool for lemmatization for RTC texts `lemma_id_closest`(txt). Click `Show code` to view details. { display-mode: "form" }

#There are differences between RTC and AES/AED transliterations.
#In order to investigate the differances, this short function
#lemma_id_closest was written. The function is generally not fast
#enough to run for all the words, but may prove useful when debugging the
#differences.
def lemma_id_closest_(txt):
  sorted = []
  unify_txt = unidecode(txt).lower()
  for key in lemma_id_dict.keys():
    unify_key = unidecode(key).lower()
    weight = Levenshtein.distance(unify_txt, unify_key)
    heapq.heappush(sorted, (weight, key))
  best = []
  for i in range(5):
    best.append(heapq.heappop(sorted))
  return best

def lemma_id_closest(txt):
  m = lemma_id_closest_(txt)
  remtxt = ''
  for char in txt: # Remove '⸢', '⸣', '[', ']' etc.
    if char not in rem_list:
      remtxt += char
  return (txt, set(m + lemma_id_closest_(remtxt)))


# RTC


In [ ]:
# @title Load Ramses Transliteration Corpus (RTC). Click `Show code` to view details. [*Ramses Automated Transliterator*](https://gitlab.cnam.fr/gitlab/rosmorse/ramses-trl) by [**Université de Liège/Projet Ramsès**](http://ramses.ulg.ac.be/) & [**Serge Rosmorduc**](http://cedric.cnam.fr/lab/en/author/Rosmorduc/) is licensed under `CC BY-NC-SA` 4.0. { display-mode: "form" }

rtc_cannot_lemmatize = {}
def process_src_line(src):
  chars = src.rstrip('\n')
  return (chars, chars.split(" "))
def process_tgt_line(dst):
  dst = dst.rstrip('\n')
  chars = dst.split()
  chars = "".join(chars)
  chars = chars.replace('_', ' ').rstrip(' ')
  return (chars, chars.split())
count_err = 0
def lemma_id_translate(token):
  s = ""
  if token is None:
    return s
  # There is difference between j/i
  if token in aecorpus.aed:
    t = aecorpus.aed[token]
    if 'translations' in t:
      tr = t['translations']
      if len(tr) > 0 and len(tr[0]) > 0:
        tr1 = tr[:1][:1]
        tr0 = list(filter(lambda translation: translation[1] == 'en', tr))
        if len(tr0) > 0:
          tr1 = tr0
        s += "/" + str(tr1[0][0])
  return s
g2u_map = GardinerToUnicodeMap()
g2u_alt_map={ # Replacements for characters missing somewhere.
    'MISSING': "\uFFFD",
    '': "\u25A1",
    'Ff1': g2u_map.to_unicode_char('Z14')
}
g2u_alt_entries = len(g2u_alt_map)
def g2u(gardiner_list):
  u = []
  for g in gardiner_list.split(" "):
    if g in g2u_alt_map:
      out_char = g2u_alt_map[g]
    else:
      out_char = g2u_map.to_unicode_char(g)
    if out_char is None:
      out_char = "\uFFFD"
      g2u_alt_map[g] = out_char
    else:
      u.append(out_char)
  return " ".join(u)

def read_rtc():
  rtc = []
  rtc_g = []
  rtc_t = []
  rtc_1 = []

  c = 0
  c_found = 0
  model_sentences = []
  full_count = 0
  gaps_count = 0
  total_count = 0
  for i in range(len(rtc_src_files)):
    sentences = []
    print("Processing " + rtc_src_files[i])
    with open(rtc_path + rtc_src_files[i] + rtc_ext, "r") as src_f:
      src_lines = src_f.readlines()
    with open(rtc_path + rtc_tgt_files[i] + rtc_ext, "r") as tgt_f:
      tgt_lines = tgt_f.readlines()
    assert len(src_lines) == len(tgt_lines)
    for src_, tgt_ in zip(src_lines, tgt_lines):
      src, src_list = process_src_line(src_)
      tgt, tgt_list = process_tgt_line(tgt_)
      sentence = {
          'text': tgt,
          'translation': '', # No full sentence translations
          'gardiner': src,
          'gardiner_list': src_list,
          'corpus': rtc_src_files[i],
          'tokens': [],
          'token_list': [], # TODO
          'sentence_transliteration': tgt,
          'gaps': False
      }
      num_found = 0
      tokens_found = True
      print_sentence=False
      # Mapping between gardiner and transliteration would be good.
      # For now the tokens are based on transliteration.
      for idx,tr in enumerate(tgt_list):
        if tr == "//":
          continue # Skip spaces
        # Construct tokens similar to ones with AES.
        tr_ = tr.replace("i", "j").replace("I", "J") # Common difference
        if tr == "---" or tr == "": # Missing symbols have different notation.
          tr_ = "LACUNA"
        trl = lemmatize(tr_)
        trid = get_lemma_id(tr)
        if trid is None:
          trid = get_lemma_id(tr_)
        if trid is None:
          trid = get_lemma_id(trl)

        if len(tr) > 0 and trid is None:
            pattern = r'[^0-9\/[]'
            if re.search(pattern, tr):
              pass
            else:
              # All numbers should be handled
              print(f"Number {tr} not handled")
              global count_err
              count_err += 1
              print_sentence = True
              if count_err > 5:
                raise "Too many errors"
        if trid is None:
          rtc_cannot_lemmatize[tr] = trl
          #print('Cannot lemmatize', tr, trl, trid)
          #print('Closest matches', lemma_id_closest(tr))
          c += 1
        else:
          num_found += 1
        token = (
            tr,
            lemma_id_translate(trid),
            trl, # Rough lemmatization.
            "" if trid is None else trid, # Use lemma id if found
            "" # TODO: Find pos.
        )
        if trid is None:
          tokens_found = False
        sentence["tokens"].append(token)
      sentence["token_list"] = list([token[3] for token in sentence["tokens"]])

      # Check if there are gaps in sentence
      # TODO: Consider partially readable symbols etc.
      if 'LACUNA' in src_list or 'LACUNA' in tgt_list:
        sentence['gaps'] = True
        tokens_found = False
        gaps_count += 1
      sentence['tokens_found'] = tokens_found
      full_count += 1 if tokens_found else 0
      total_count += 1
      if sentence['gaps'] == False and tokens_found: # previously: (tokens_found or num_found > 3):
        if c_found < 5:
          model_sentences.append(sentence)
        if len(model_sentences) < 7 and '5' in sentence['sentence_transliteration'] and '1/8' in sentence['sentence_transliteration']:
          model_sentences.append(sentence)
      c_found += 1
      if sentence['gaps']:
        rtc_g.append(sentence)
      elif i == 1: # Validation set
        rtc.append(sentence)
        rtc_t.append(sentence)
      else:
        rtc.append(sentence)
        rtc_1.append(sentence)
      if print_sentence:
        print("Debug sentence:")
        print("  Text:", sentence["sentence_transliteration"])
        print("  Token_list", sentence["token_list"])
        print("  Gardiner:", sentence["gardiner"])
        print("  Unicode:", g2u(sentence["gardiner"]))
        print_sentence = False

  print("Model sentences")
  for idx, sentence in enumerate(model_sentences):
    print(idx+1)
    print("  Text:", sentence["sentence_transliteration"])
    print("  Token_list", sentence["token_list"])
    #print("  Tokens", sentence["tokens"])
    print("  Gardiner:", sentence["gardiner"])
    print("  Unicode:", g2u(sentence["gardiner"]))

  if g2u_alt_entries < len(g2u_alt_map):
    print(f"Unmapped Gardiner symbols: {len(g2u_alt_map) - g2u_alt_entries}")


  print()

  print(f"All Sentences: {total_count}; fully lemmatized and no gaps: {full_count}")
  #print(f"gapped: {gaps_count}") (already on the next line)
  return (rtc, rtc_1, rtc_t, rtc_g)

print("Reading and processing RTC.")
rtc = read_rtc()
aecorpus.rtc = rtc[0]
aecorpus.rtc_1 = rtc[1]
aecorpus.rtc_t = rtc[2]
aecorpus.rtc_g = rtc[3]
print(f"RTC partitioned: usable sentences={len(aecorpus.rtc)} incl: training={len(aecorpus.rtc_1)} test={len(aecorpus.rtc_t)} gapped: {len(aecorpus.rtc_g)}")

Reading and processing RTC.
Processing src-train
Processing src-val
Model sentences
1
  Text: iw iw =i r swr m =f
  Token_list ['21881', '21881', '10030', '91900', '130360', '64360', '10050']
  Gardiner: M17 Z7 M17 Z7 A1 D21 S29 G36 D21 N35A A2 M17 G17 I9 
  Unicode: 𓇋 𓏲 𓇋 𓏲 𓀀 𓂋 𓋴 𓅨 𓂋 𓈗 𓀁 𓇋 𓅓 𓆑 □
2
  Text: pA hrw n ms pA inw
  Token_list ['851446', '99060', '78870', '74700', '851446', '850830']
  Gardiner: G41 G1 O4 D21 Z7 N5 Z1 N35 MISSING G41 G1 W25 N35 W24 Z7 Y1 Z2 
  Unicode: 𓅯 𓄿 𓉔 𓂋 𓏲 𓇳 𓏤 𓈖 � 𓅯 𓄿 𓏎 𓈖 𓏌 𓏲 𓏛 𓏥 □
3
  Text: pA sS XAr 1/8 1/32 1/40 r 5 1/3
  Token_list ['851446', '144360', '122580', '850814', '850814', '850814', '91900', '850814', '850814']
  Gardiner: G41 G1 Y3 A1 V19 D11 F16 D21 V20 V20 V20 V20 D21 Z2 2 D21 Z2 
  Unicode: 𓅯 𓄿 𓏞 𓀀 𓎅 𓂁 𓄏 𓂋 𓎆 𓎆 𓎆 𓎆 𓂋 𓏥 𓂋 𓏥 □
4
  Text: s 50 6 wa nb XAr ir.w n XAr 10 2 1/8
  Token_list ['147350', '850814', '850814', '400101', '81650', '122580', '851809', '78870', '122580', '850814', '850814', '850814']
  Gardiner: O34 A1 V20 V20 V20 V20 V

In [ ]:
# @title Details: Transliterations that could not be lemmatized. Click `Show code` to view details.
# @markdown RTC transliterations use slightly different characters than AES. Therefore many of them cannot be processed yet.
# @markdown This cell allows to examine the trouble some ones. Note: Currently we use transliteration and not lemmatized forms for processing. Making issues non-critical.
from ipywidgets import widgets
from IPython.display import clear_output

def create_expandable_table(pagesize=10):
  df = pd.DataFrame({ 'Transliteration': list(rtc_cannot_lemmatize.keys()),
                      'Transformed': list(rtc_cannot_lemmatize.values()) })

  def on_button_clicked(event):
    page=0
    if 'new' in event:
      page=event['new']
    keys = list(rtc_cannot_lemmatize.keys())
    values = list(rtc_cannot_lemmatize.values())
    keys = keys[page*pagesize:(page+1)*pagesize]
    values = values[page*pagesize:(page+1)*pagesize]
    s = []
    for key in tqdm_notebook(keys, desc="Open page"): # Process first page
      s.append(f'{lemma_id_closest(key)}')
    pd.set_option('display.max_rows', pagesize)
    df = pd.DataFrame({ 'Transliteration': keys,
                        'Transformed': values,
                        'Closest matches': s })
    clear_output(wait=True)
    display(df)
    int_range = widgets.IntSlider(min=0, max=(len(rtc_cannot_lemmatize) + pagesize - 1)//pagesize, value=page, name="Page")
    int_range.observe(on_button_clicked, names='value')
    display(int_range)
    pd.set_option('display.max_rows', 5)

    # display(btn) # No longer show more info button.

  btn = widgets.Button(description="More Info\u2026", layout=widgets.Layout(width='100px'))
  btn.on_click(lambda event: on_button_clicked({}))

  display(df)
  display(btn)

create_expandable_table()

,Transliteration,Transformed
0,ns-imn,nz-jmn
1,wsr-mAa.t-ra,wzr-mꜣꜥ.t-rꜥ
...,...,...
6200,+tpAy.s[]+l,+tpꜣy.z+l
6201,mna.t-[,mnꜥ.t-


Button(description='More Info…', layout=Layout(width='100px'), style=ButtonStyle())

# [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html)

Generate Word2Vec and FastText models, test them and allow user to view some results.

The models are implemented using gensim libraries.


In [ ]:
# @title Details: Helper functions. Click `Show code` to view details. { display-mode: "form" }
# @markdown Helper functions are useful to make actual model generation functions easy to read.
# @markdown `print_sentence`, `token_to_text`, `start_pad`, `end_pad`, `is_pad`, `has_pad`, `token_seq`, `token_seq_tr`, `token_seq_rtc`
def print_sentence(sentence):
  print(f'{sentence["text"]}')
  print(f'. tokens: {sentence["token_list"]}')
  print(f'. transliteration: {sentence["sentence_transliteration"]}')
  print(f'. translation: {sentence["translation"]}')
  # Optional English translation (just one sentence here)
  # The sentence dictionary for German to English translations can be used for more.
  if sentence["sentence_transliteration"] == "bn ftt =tw =f":
    print('. It (the inscription on the stele) should not be erased.') # Google Translate (from de)
  elif 'en' in sentence: # Does dictionary lookup translation exist?
    print(f'. en: {sentence["en"]}')

def token_to_text(token):
  s = token
  if token in aecorpus.aed:
    t = aecorpus.aed[token]
    if 'form' in t:
      s += "/" + t['form']
    if 'translations' in t:
      tr = t['translations']
      if len(tr) > 0 and len(tr[0]) > 0:
        tr1 = tr[:1][:1]
        tr0 = list(filter(lambda translation: translation[1] == 'en', tr))
        if len(tr0) > 0:
          tr1 = tr0
        s += "/" + str(tr1[0][0])
  return s

def start_pad(padlen=2):
  if padlen == 2:
    return ['<s0>', '<s1>']
  return [f'<s{i}>' for i in range(padlen)]

def end_pad(padlen=2):
  if padlen == 2:
    return ['</s1>', '</s0>']
  return [f'</s{padlen-i-1}>' for i in range(padlen)]

def is_pad(x):
  return x in start_pad(9) + end_pad(9)

def has_pad(a):
  for x in a:
    if is_pad(x):
      return True
  return False

def token_seq(sentence, replace_numbers=False, padlen=2, nopad=False):
    # Note: replace numbers is no-op here. replacement has already been performed
    token_seq = [token[3] for token in sentence['tokens']] # With lemma id
    if nopad:
      return token_seq
    return start_pad(padlen) + token_seq + end_pad(padlen)

def token_seq_tr(sentence, replace_numbers=False, padlen=2, nopad=False):
    # Note: replace numbers is no-op here. replacement has already been performed
    token_seq = sentence["sentence_transliteration"].split(" ") # Before lemmatization
    if nopad:
      return token_seq
    return start_pad(padlen) + token_seq + end_pad(padlen)

re_fraction=re.compile('^[0-9]+\/[0-9]+$')
def token_seq_rtc(sentence, replace_numbers=False, padlen=2, nopad=False):
    try:
      token_seq = [token[0] for token in sentence['tokens']] # With text - for RTC currently best pick
    except:
      print("Error with: ", sentence)
      raise "Errorred"
    if replace_numbers:
      # Handle numbers
      global re_fraction
      for idx in range(len(token_seq)):
        if token_seq[idx].isnumeric():
          token_seq[idx] = "1...n"
        if re_fraction.match(token_seq[idx]):
          token_seq[idx] = "1...n"
    #token_seq = [token[3] for token in sentence['tokens']] # With lemma id
    if nopad:
      return token_seq
    return start_pad(padlen) + token_seq + end_pad(padlen)


To work in beginning or ending of sentences padding may be useful.
Two pads seemed to work better than 1.

In [ ]:
# @title Prediction statistics -- `PredictionStats`. Click `Show code` in the code cell. { display-mode: "form" }
# @markdown
class PredictionStats:
    """Collect statistics regarding correctness of tests"""

    def __init__(self, name):
      self.name = name
      self.c = Counter()
      self.fails = 0
      self.start = time.time()
      self.end = float("nan")

    def start_testing(self):
      self.start = time.time()
      return self
    def end_testing(self):
      self.end = time.time()
      return self
    def miss(self):
      self.c[-1] += 1
    def fail(self):
      self.fails += 1   # Count failures outside N
    def predicted(self, item, predictions):
      if item in predictions:
        idx = predictions.index(item) + 1
        self.c[idx] += 1
      else:
        self.miss()
        idx = -1
      return idx
    def get_name(self):
      return self.name
    def get_n(self):
      # Failures are counted in n.
      return self.c.total() + self.fails
    def get_heading(self, include_mrr5=False):
      if include_mrr5:
        return ['hit', 'hit@5', 'hit@10', 'missed', 'MRR@5', 'MRR@10', 'N', 'untestable', 'ms/test']
      return ['hit', 'hit@5', 'hit@10', 'missed', 'MRR', 'N', 'untestable', 'ms/test']
    def get_stats(self, include_mrr5=False):
      accu = 0
      accu_ = 0
      accu5 = 0
      accu10 = 0
      for r in range(1, 6):
        accu_ += self.c[r]/r
      for r in range(1, 11):
        accu += self.c[r]/r
        accu5 += self.c[r] if r <= 5 else 0
        accu10 += self.c[r]
      n = self.get_n()
      if n < 1:
        n = 1
      if include_mrr5:
        return [
          self.c[1] / n,
          accu5 / n,
          accu10 / n,
          self.c[-1] / n,
          accu_ / n,
          accu / n,
          n,
          self.fails,
          ((self.end - self.start) * 1000) / n
        ]
      return [
          self.c[1] / n,
          accu5 / n,
          accu10 / n,
          self.c[-1] / n,
          accu / n,
          n,
          self.fails,
          ((self.end - self.start) * 1000) / n
      ]

# These variables collect statistics
prs = [None] * 2
stats = dict()

In [ ]:
# @title Prediction Function `predict_words`. Use model(s) to predict word. Click `Show code` in the code cell. { display-mode: "form" }
# @markdown Prediction function has restrictions as follows:
# @markdown
# @markdown - It can only predict one part of text.
# @markdown - It can only work on single word missing parts.

def predict_words(tokens,model,model_score=None,lacuna="LACUNA",verbose=False,omitted=None,vocab=None,algo=0,topn=10,padlen=2,stat_append=None,stat_id=None,nopad=False):
  """Predict missing word(s) from sentence.

    Parameters
    ------------
        tokens: list of str
            Sentence (token sequence) to predict (including start_pad() and end_pad())
        model: Word2Vec model
            Model used for predicting with predict_output_word
        model_score: Word2Vec model
            Model used for scoring with score
        lacuna: str
            Placeholder for missing word
        verbose: bool
            Perform debug prints (default=False)
        omitted: str
            Provide omitted token (only needed for verbose mode debug prints)
        vocab: list of str or None
            Possible words - if none, all words are possible. (only for algo=2)
        algo: int [-1, 2]
            Select algorithm to perform prediction
        topn: int
            Select how many result to return (default=10)

    Return
    -----------
        predictions: list of tuples (str, float)
            List of predictions in the best first order with weights.

        May raise exceptions on exceptional situations.

    Bugs
    ------------
        current implementation is able to predict only one missing word.
  """
  if vocab is None:
    if hasattr(model, 'wv'):
      vocab = model.wv.index_to_key

  try:
    idx = tokens.index(lacuna)
  except ValueError:
    return [(1.0), " ".join(tokens)]*topn

  predict = predict_words_internal(idx, tokens,model,model_score,vocab,algo,topn,padlen,nopad)
  if verbose:
    print(f"OMITTED = {omitted}")
    for i, p in enumerate(predict):
      if p[0] == omitted:
        print(f"PREDICTED {i} {token_to_text(p[0])} (prob: {p[1]}) match [rank={i+1}]")
      else:
        if hasattr(model, 'wv'):
          similarity = model.wv.n_similarity(omitted, p[0])
        else:
          similarity = 1.0 - i / topn # No similarity function
        print(f"PREDICTED {i} {token_to_text(p[0])} (prob: {p[1]}) similarity with orig: {similarity}")
  if stat_append is not None:
    tokens_ = filter(lambda t: not has_pad(t), tokens)
    simi=[]
    hit_idx = None
    for i, p in enumerate(predict):
      if hasattr(model, 'wv'):
        similarity = model.wv.n_similarity(omitted, p[0])
      else:
        similarity = 1.0 - i / topn # No similarity function
      simi.append([p[0],p[1],similarity])
      if similarity == 1:
        hit_idx = i + 1
    stat_append.append((stat_id, tokens_, omitted, hit_idx, tuple([p[0] for p in predict]), simi))
  return predict

# Underlying worker function
def predict_words_internal(idx, tokens,model,model_score=None,vocab=None,algo=0,topn=10,padlen=2,nopad=False):
  token_seq_before = tokens[:idx]
  token_seq_after = tokens[idx + 1:]

  if algo == -1:
    # Simply predict top vocabulary words.
    i = 0
    while is_pad(vocab[i]):
      i = i + 1
    return [(vocab[i+l], 1/(l+1)) for l in range(0, 10)]
  if algo == 0:
    if len(token_seq_before) > 0 and len(token_seq_after) > 0:
      if len(token_seq_before) > len(token_seq_after):
        token_seq_before = token_seq_before[-len(token_seq_after):]
      elif len(token_seq_before) < len(token_seq_after):
        token_seq_after = token_seq_after[:len(token_seq_before)]
      maxpad=4
      predict = model.predict_output_word(token_seq_before + token_seq_after,topn=topn+maxpad)
      # Filter predictions, remove padding symbols
      predict_out = []
      for x in predict:
        if not is_pad(x[0]):
          predict_out.append(x)
        if len(predict_out) == topn:
          break
      return predict_out
    else:
      return ValueError("Input not supported: no before or after tokens")
  if algo == 1:
    if len(token_seq_before) > 0 and len(token_seq_after) > 0:
      if len(token_seq_before) > len(token_seq_after):
        token_seq_before = token_seq_before[-len(token_seq_after):]
      elif len(token_seq_before) < len(token_seq_after):
        token_seq_after = token_seq_after[:len(token_seq_before)]
      predict = model.predict_output_word(token_seq_before + token_seq_after,topn=topn*10)
      sentences = []
      for pr in predict:
        tokens = token_seq_before + [pr[0]] + token_seq_after
        if nopad:
          tokens = tokens[len(start_pad(padlen)):len(tokens) - len(end_pad(padlen))]
        sentences.append(" ".join(tokens))
      scores = model_score.score(sentences)
      adjusted_predictions=[(predict[i][0], scores[i]) for i in range(len(scores))]
      adjusted_predictions.sort(key = lambda x: -x[1])
      predict = adjusted_predictions[:topn]
      return predict
    else:
      return ValueError("Input not supported: no before or after tokens")
  if algo == 2:
    sentences = []
    for token in vocab:
      tokens = token_seq_before + [token] + token_seq_after
      if nopad:
        tokens = tokens[len(start_pad(padlen)):len(tokens) - len(end_pad(padlen))]
      sentences.append(" ".join(tokens))
    scores = model_score.score(sentences)
    adjusted_predictions=[(vocab[i], scores[i]) for i in range(len(scores))]
    # TODO: Quicker sorting here
    adjusted_predictions.sort(key = lambda x: -x[1])
    predict = adjusted_predictions[:topn]
    return predict
  raise ValueError("algo is not supported")



In [ ]:
# @title Define Model Generation Progress Class. Click `Show code` in the code cell. { display-mode: "form" }
# @markdown

class TqdmModelProgress(CallbackAny2Vec):
     '''Callback to log information about training'''

     def __init__(self, epochs):
         self.epoch = 0
         self.epochs = epochs
         self.tqdm = tqdm_notebook(desc="Starting", total=self.epochs,
                                   unit="epoch", leave=False)

     def on_epoch_begin(self, model):
         if self.epoch == 0:
             self.tqdm.desc = "Training"

     def on_epoch_end(self, model):
         self.epoch += 1
         if self.epoch == self.epochs:
             self.tqdm.description = "Finished"
         loss = model.get_latest_training_loss()
         if loss is not None:
             self.tqdm.set_postfix({'loss=': loss})
         self.tqdm.update(1)

     def close(self):
         self.tqdm.close()

In [ ]:
# @title Define Model Testing Functions. Click `Show code` in the code cell. { display-mode: "form" }
# @markdown `test_w2v` for testing AES with lemma id. Use `seq` parameter to override `token_seq`to test AES/TR or RTC.
# @markdown Alternatively you may use convenience wrappers `test_w2v_tr` and `test_w2v_rtc`.
def test_w2v(name, w2v, sentences, silent=False, padlen=2, algo=0, seq=token_seq, omitted_indexes=None):
  pr = PredictionStats(name)
  spadlen = padlen
  fpadlen = padlen * 2
  random.seed(42)
  if silent == False:
    print("Processing test sentences with single hidden word")
  c=0
  idx=0
  for sentence in sentences:
    tokens = seq(sentence,padlen=padlen)
    if len(tokens) == fpadlen:
      pr.fail() # Failed processing
      continue
    if omitted_indexes is None:
      token_omitted = random.randrange(len(tokens) - fpadlen) + spadlen
    else:
      token_omitted = omitted_indexes[idx] + spadlen
      idx = idx + 1
    token_orig = tokens[token_omitted]
    tokens[token_omitted] = "LACUNA"
    verbose = False
    if c < 5 and silent == False:
      verbose = True
    if c == 5 and silent == False:
        print("Processing remaining sentences quietly")
    c += 1

    try:
      predict = predict_words(tokens, w2v, None, verbose=verbose, omitted=token_orig, algo=algo, padlen=padlen)
      if len(predict) == 0:
        raise Exception("No predictions")
      try:
        pr.predicted(token_orig, [x[0] for x in predict])
      except:
        pr.fail() # Failed processing predictions
    except:
      pr.fail() # Failed processing predicting

  return pr.end_testing()

def test_w2v_tr(name, w2v, sentences, silent=False, padlen=2, algo=0):
  return test_w2v(name, w2v, sentences, silent, padlen, algo, seq=token_seq_tr)

def test_w2v_rtc(name, w2v, w2v_hs=None, algo=0, sentences=aecorpus.rtc_t, padlen=2, silent=False, nopad=False):
  return test_w2v(name, w2v, sentences, silent, padlen, algo, seq=token_seq_rtc)



In [ ]:
# @title Model Training function `get_gensim_gen`. Click `Show code` in the code cell. { display-mode: "form" }

# @markdown Function allows automatic storage and retrieval of models.
# @markdown
# @markdown Update model id when inputs or model settings are updated.
#
gensim_model_prefix="2024-03-13a" # @param {type:"string"}
gensim_model_store = True # @param {type:"boolean"}

def get_gensim_gen(corpus, vector_size=200, epochs=100, padlen=2,
                   silent=False, token_seq_func = token_seq, nobar=False,
                   model = gensim.models.Word2Vec, id=None,
                   noload=False, nosave=False, test=None, testlimit=0.1,
                   prev_model = None,
                   **kwargs):
  global gensim_model_id
  global gensim_model_store
  if id is None:
    id = model.__class__.__name__
  filepath = f"models/{gensim_model_prefix}-{id}.model"
  if gensim_model_store and not noload and os.path.exists(filepath):
    model_out = model.load(filepath)
    if test is not None:
      if test(model_out) < testlimit:
        raise Exception("Loaded Model failed to validate")
    return model_out

  c, cn, cn2 = (0, 0, 0)
  sentences_list = []
  token_kwargs = {key:kwargs[key] for key in ['nopad', 'replace_numbers'] if key in kwargs}
  for sentence in corpus:
    tokens = token_seq_func(sentence, padlen=padlen, **token_kwargs)
    if silent == False:
      if c < 5:
        print(tokens)
        c += 1
      elif cn < 2 and '1...n' in tokens:
        print(tokens)
        cn += 1
      elif cn2 < 2 and '123' in tokens:
        print(tokens)
        cn2 += 1
    sentences_list.append(tokens)
  in_kwargs = dict(kwargs)
  if 'window' not in in_kwargs:
    in_kwargs['window']=5
  if 'min_count' not in in_kwargs:
    in_kwargs['min_count']=1
  if 'replace_numbers' in in_kwargs:
    del in_kwargs['replace_numbers']
  if 'nopad' in in_kwargs:
    del in_kwargs['nopad']
  callbacks = []
  if not nobar:
    callbacks = [TqdmModelProgress(epochs)]

  if prev_model is None:
    model_out = model(sentences=sentences_list, vector_size=vector_size,
                      workers=10, epochs=epochs, callbacks=callbacks,
                      **in_kwargs)
  else:
    #model_out = prev_model.train(sentences=sentences_list, vector_size=vector_size,
    #                  workers=10, epochs=epochs-prev_model.epochs, callbacks=callbacks,
    #                  **in_kwargs)
    # Some arguments only apply in creation of model.
    if 'vector_size' in in_kwargs:
      del in_kwargs['vector_size']
    model_out = prev_model.train(corpus_iterable=sentences_list,
                      epochs=epochs, callbacks=callbacks,
                      **in_kwargs)

  if test is not None:
    if test(model_out) < testlimit:
      raise Exception("Trained model failed to validate")
    if gensim_model_store and not nosave:
      model_out.save(filepath)
  callbacks[0].close()
  return model_out

def get_gensim_ft_rtc(corpus, **kwargs):
  return get_gensim_gen(corpus, model=gensim.models.FastText,
                        token_seq_func = token_seq_rtc, **kwargs)

def get_gensim_ft_aes(corpus, **kwargs):
  return get_gensim_gen(corpus, model=gensim.models.FastText,
                        token_seq_func = token_seq_tr, **kwargs)




In [ ]:
# @title Train models and perform testing. Click `Show code` in the code cell. { display-mode: "form" }
# @markdown The trained Word2Vec models are stored on Google drive.
# @markdown Optional trained model is FastText (FT) model. It cannot be
# @markdown stored, but are always retrained. All models (loaded and trained)
# @markdown are tested and test results are stored for later processing or display.
# @markdown
# @markdown When all models are retrained expect execution of the cell to take
# @markdown around 20 minutes. If only FastText models are retrained, expect
# @markdown execution to succeed in 6-7 minutes.
# @markdown If no models are retrained this stage runs quickly.

train_ft=False # @param {type:"boolean"}

def store_stats(pr):
  stats[pr.get_name()] = pr
  values = pr.get_stats()
  return values[4]
for x in tqdm_notebook(range(7), desc="Model", leave=False):
  if x == 0:
    opts_known_good_150 = {'ns_exponent': 0.21821168418246245, 'negative': 21, 'sg': 1,
                   'vector_size': 227, 'window': 2, 'min_count': 4,
                   'shrink_windows': False, 'compute_loss': True, 'epochs': 150}
    w2v = get_gensim_gen(aecorpus.aes_1, token_seq_func = token_seq,
                         silent=True, id="W2V-AES-alt",
                         test=lambda model: store_stats(test_w2v("W2V AES", model, aecorpus.aes_t, silent=True)),
                         **opts_known_good_150) # Takes around 2 minutes
  if x == 1:
    opts_nontested_p0 = {'ns_exponent': 0.21821168418246245, 'negative': 21, 'sg': 1,
                         'vector_size': 227, 'window': 2, 'min_count': 1,
                         'shrink_windows': False, 'compute_loss': True, 'epochs': 15}
    w2vp0 = get_gensim_gen(aecorpus.aes_1, token_seq_func = token_seq,
                           silent=True, padlen=0, nopad=True, id="W2V-AES-p0v1.2",
                           test=lambda model: store_stats(test_w2v("W2V-p0 AES", model, aecorpus.aes_t, padlen=0, silent=True)),
                           **opts_nontested_p0) # Takes around 2 minutes
  if x == 2:
    opts_good = {'ns_exponent': 0.21821168418246245, 'negative': 21, 'sg': 1,
                   'vector_size': 227, 'window': 2, 'min_count': 4,
                   'shrink_windows': False, 'compute_loss': True, 'epochs': 15}

    w2vp1 = get_gensim_gen(aecorpus.aes_1, token_seq_func = token_seq,
                           silent=True, padlen=1, id="W2V-AES-p1",
                           test=lambda model: store_stats(test_w2v("W2V-p1 AES", model, aecorpus.aes_t, padlen=1, silent=True)),
                           **opts_good) # Takes around 2 minutes
  if x == 3:
    opts_good_p3 = {'ns_exponent': 0.30145585, 'negative': 26, 'sg': 1, 'vector_size': 172, 'window': 2, 'shrink_windows': False, 'compute_loss': True, 'epochs': 15}
    w2vp3 = get_gensim_gen(aecorpus.aes_1, token_seq_func = token_seq,
                           silent=True, padlen=3, id="W2V-AES-p3",
                           test=lambda model: store_stats(test_w2v("W2V-p3 AES", model, aecorpus.aes_t, padlen=3, silent=True)),
                           **opts_good_p3) # Takes around 2 minutes
  if x == 4:
    #opts_good_rtc = {
    #  'ns_exponent': 0.17955186844534993, 'negative': 22, 'sg': 1,
    #  'vector_size': 259, 'window': 2, 'min_count': 2,
    #  'epochs': 15}
    #w2v_rtc = get_gensim_gen(aecorpus.rtc_1, token_seq_func = token_seq_rtc, silent=True, id="W2V-RTC",
    #                         test=lambda model: store_stats(test_w2v_rtc("W2V RTC", model, None, 0, aecorpus.rtc_t, silent=True)),
    #                         **opts_good_rtc) # Takes around 3 minutes
    optimized_w2v_opts = {'ns_exponent': 0.05, 'negative': 21, 'sg': 0, 'vector_size': 200,
                      'window': 2, 'sample': 0.01, 'shrink_windows': True,
                      'compute_loss': True, 'epochs': 150, 'compute_loss': True}
    w2v_rtc = get_gensim_gen(aecorpus.rtc_1, token_seq_func = token_seq_rtc, silent=True, id="W2V RTC-optimized",
                             test=lambda model: store_stats(test_w2v_rtc("W2V RTC", model, None, 0, aecorpus.rtc_t, silent=True)),
                             **optimized_w2v_opts) # Takes around 20 minutes
  if x == 5 and train_ft:
    opts_good_ft = {'ns_exponent': 0.21821168418246245, 'negative': 21,
                    'sg': 1, 'vector_size': 227, 'window': 2, 'min_count': 4,
                    'shrink_windows': False, 'silent': True, 'epochs': 15}
    ft_rtc = get_gensim_ft_rtc(aecorpus.rtc_1, id="ft_rtc", nosave=True, noload=True,
                               test=lambda model: store_stats(test_w2v_rtc("FT RTC", model, None, 0, aecorpus.rtc_t, silent=True)),
                               **opts_good_ft) # Takes around 1,5 minutes
    #todo: try more variants like
    #ft_rtc_nopad = get_gensim_ft_rtc(aecorpus.rtc_1, id="ft_rtc", nopad=True, **opts_good_ft) # Takes around 1,5 minutes
  if x == 6:
    opts_aes_tr_150 = {'ns_exponent': 0.21821168418246245, 'negative': 21, 'sg': 1,
                       'vector_size': 227, 'window': 2, 'min_count': 4,
                       'shrink_windows': False, 'compute_loss': True, 'epochs': 150}
    w2v_tr = get_gensim_gen(aecorpus.aes_1, token_seq_func = token_seq_tr,
                         silent=True, id="W2V-AES-TR-alt",
                         test=lambda model: store_stats(test_w2v_tr("W2V AES TR", model, aecorpus.aes_t, silent=True)),
                         **opts_aes_tr_150) # Takes around 20 minutes

print("Trained Models are now available: " + (", ".join(stats.keys())))


Model:   0%|          | 0/7 [00:00<?, ?it/s]

Trained Models are now available: W2V AES, W2V-p0 AES, W2V-p1 AES, W2V-p3 AES, W2V RTC, W2V AES TR


In [ ]:
#Check variants
additional_experimental_models = False
out = widgets.Output(layout={'border': '1px solid black'})
display(out)

def stats_show(stats):
  row_names = stats.keys()
  ps = [pr.get_stats(include_mrr5=True) for pr in stats.values()]
  column_names = [ '&', 'hit@1', '&', 'hit@5', '&', 'hit@10', '&', 'MRR@10', '&', 'Processed sentences', '\\\\' ]
  dfdata = np.row_stack([('&', f'{x[0]:.6f}', '&', f'{x[1]:.6f}', '&', f'{x[2]:.6f}', '&', f'{x[5]:.6f}', '&', f'{x[6]}', '\\\\') for x in ps])
  pd.set_option('display.max_rows', 1000)
  pd.set_option('display.min_rows', 1000)
  text_stats_df = pd.DataFrame(dfdata, columns=column_names, index=row_names)
  text_stats_df.index = row_names
  out.clear_output(wait=True)
  with out:
    print(text_stats_df)

stats_new = dict()
def store_stats_print(pr):
  stats[pr.get_name()] = pr
  stats_new[pr.get_name()] = pr
  values = pr.get_stats()
  stats_show(stats_new)
  return values[4]

# Optimized parameters from two round optimization process with smallers sentence sets.
optimized_w2v_opts = {'ns_exponent': 0.05, 'negative': 21, 'sg': 0, 'vector_size': 200,
                      'window': 2, 'sample': 0.01, 'shrink_windows': True,
                      'compute_loss': True, 'epochs': 150, 'compute_loss': True}

optimized_w2v_sg_opts = {'ns_exponent': 0.05, 'negative': 21, 'sg': 1, 'vector_size': 200,
                         'window': 2, 'sample': 0.01, 'shrink_windows': True,
                         'compute_loss': True, 'epochs': 150, 'compute_loss': True}

if additional_experimental_models:
  # Compare against basic models and previous experimental models
  for x in ["W2V AES TR", "W2V AES TR-alt",
            "W2V AES TR-optimized",
            "W2V AES TR-optimized-sg",
            "W2V AES", "W2V AES-alt", "W2V AES-optimized", "W2V AES-optimized-sg",
            "W2V RTC", "W2V RTC-alt", "W2V RTC-optimized", "W2V RTC-optimized-sg"]:
    if x in stats:
      store_stats_print(stats[x])
  for x in tqdm_notebook(range(7, 8), desc="Model", leave=False):
    if x == 2:
      opts_known_good_150 = {'ns_exponent': 0.21821168418246245, 'negative': 21, 'sg': 1,
                   'vector_size': 227, 'window': 2, 'min_count': 4,
                   'shrink_windows': False, 'compute_loss': True, 'epochs': 150}
      w2v_alt = get_gensim_gen(aecorpus.aes_1, token_seq_func = token_seq,
                           silent=True, id="W2V-AES-alt",
                           test=lambda model: store_stats_print(test_w2v("W2V AES-alt", model, aecorpus.aes_t, silent=True)),
                           **opts_known_good_150) # Takes around 20 minutes
    if x == 6:
      w2v_alt2 = get_gensim_gen(aecorpus.aes_1, token_seq_func = token_seq,
                           silent=True, id="W2V-AES-optimized",
                           test=lambda model: store_stats_print(test_w2v("W2V AES-optimized", model, aecorpus.aes_t, silent=True)),
                           **optimized_w2v_opts) # Takes around 20 minutes
    if x == 7:
      w2v_alt3 = get_gensim_gen(aecorpus.aes_1, token_seq_func = token_seq,
                           silent=True, id="W2V-AES-optimized-sg",
                           test=lambda model: store_stats_print(test_w2v("W2V AES-optimized-sg", model, aecorpus.aes_t, silent=True)),
                           **optimized_w2v_sg_opts) # Takes around 20 minutes
    if x == 1:
      opts_good_rtc = {
        'ns_exponent': 0.17955186844534993, 'negative': 22, 'sg': 1,
        'vector_size': 259, 'window': 2, 'min_count': 2,
        'epochs': 150}
      w2v_rtc_alt = get_gensim_gen(aecorpus.rtc_1, token_seq_func = token_seq_rtc, silent=True, id="W2V-RTC-alt",
                                   test=lambda model: store_stats_print(test_w2v_rtc("W2V RTC-alt", model, None, 0, aecorpus.rtc_t, silent=True)),
                                   **opts_good_rtc) # Takes around 20 minutes
    if x == 4:
      w2v_rtc_alt2 = get_gensim_gen(aecorpus.rtc_1, token_seq_func = token_seq_rtc, silent=True, id="W2V-RTC-optimized",
                                   test=lambda model: store_stats_print(test_w2v_rtc("W2V RTC-optimized", model, None, 0, aecorpus.rtc_t, silent=True)),
                                   **optimized_w2v_opts) # Takes around 20 minutes
    if x == 5:
      w2v_rtc_alt3 = get_gensim_gen(aecorpus.rtc_1, token_seq_func = token_seq_rtc, silent=True, id="W2V-RTC-optimized-sg",
                                   test=lambda model: store_stats_print(test_w2v_rtc("W2V RTC-optimized-sg", model, None, 0, aecorpus.rtc_t, silent=True)),
                                   **optimized_w2v_sg_opts) # Takes around 20 minutes
    if x == 0:
      opts_aes_tr_150 = {'ns_exponent': 0.21821168418246245, 'negative': 21, 'sg': 1,
                   'vector_size': 227, 'window': 2, 'min_count': 4,
                   'shrink_windows': False, 'compute_loss': True, 'epochs': 150}
      w2v_tr_alt = get_gensim_gen(aecorpus.aes_1, token_seq_func = token_seq_tr,
                         silent=True, id="W2V-AES-TR-alt",
                           test=lambda model: store_stats_print(test_w2v_tr("W2V AES TR-alt", model, aecorpus.aes_t, silent=True)),
                           **opts_aes_tr_150) # Takes around 20 minutes
    if x == 3:
      w2v_tr_alt3 = get_gensim_gen(aecorpus.aes_1, token_seq_func = token_seq_tr,
                           silent=True, id="W2V-AES-TR-optimized",
                           test=lambda model: store_stats_print(test_w2v_tr("W2V AES TR-optimized", model, aecorpus.aes_t, silent=True)),
                           **optimized_w2v_sg_opts) # Takes around 20 minutes



Output(layout=Layout(border='1px solid black'))




# Use marete

There are combined transliterations, using both AES and Ramses sentences.
These use slightly different transliteration than AES MdC so use different models.

In [ ]:
# The cell needs to be cleaned up, but works.
# Remove exact same sentences from testing data than
# are present in training data. Default is false as
# some overlap is to be expected.
filter_duplicates=False # @ param {type:"boolean"}
w2v_tr_combined_variants=1 # @ param{type:"int"}

def sentence_transform_in(sentence):
  return {"sentence_transliteration": sentence}

def token_seq_out(sentence, replace_numbers=False, padlen=2, nopad=False):
    # Note: replace numbers has not been implemented
    token_seq = sentence["sentence_transliteration"].split(" ")
    if nopad:
      return token_seq
    return start_pad(padlen) + token_seq + end_pad(padlen)

# Read test materials
with open("snlp-project/data/marete-ramses/aligned/combined_dev.txt", "r") as train_file:
  marete_train_file_lines = train_file.readlines()
  marete_train_file_lines = [x.replace("\n", "") for x in marete_train_file_lines]

with open("snlp-project/data/marete-ramses/aligned/combined_test.txt", "r") as test_file:
  marete_test_file_lines = test_file.readlines()
  marete_test_file_lines = [x.replace("\n", "") for x in marete_test_file_lines]

marete_train_sentences_in = [sentence_transform_in(sentence) for sentence in marete_train_file_lines]
if filter_duplicates:
  marete_test_sentences_in = [sentence_transform_in(sentence) for sentence in marete_test_file_lines
                                if sentence not in marete_train_file_lines]
else:
  marete_test_sentences_in = [sentence_transform_in(sentence) for sentence in marete_test_file_lines]

for x in marete_train_file_lines:
  if '\n' in x:
    print("Found linefeed in marete_train_file_lines")
  if '\xA0' in x:
    print("Found nonbreaking space in marete_train_file_lines")
for x in marete_test_file_lines:
  if '\n' in x:
    print("Found linefeed in marete_test_file_lines")
  if '\xA0' in x:
    print("Found nonbreaking space in marete_test_file_lines")

filtered_tests = len(marete_test_file_lines) - len(marete_test_sentences_in)
print(f"Using {len(marete_train_sentences_in)} training sentences and {len(marete_test_sentences_in)} test sentences. {filtered_tests} filtered out.")

#reduce inputs
#marete_train_sentences_in_q=marete_train_sentences_in[::10]
#marete_test_sentences_in_q=marete_test_sentences_in[::10]

marete_train_sentences_in_q=marete_train_sentences_in[::10]
marete_test_sentences_in_q=marete_test_sentences_in[::10]
#marete_train_sentences_in_q=marete_train_sentences_in[::10]
#marete_test_sentences_in_q=marete_test_sentences_in[::10]
#marete_train_sentences_in_q=marete_train_sentences_in
#marete_test_sentences_in_q=marete_test_sentences_in

def stats_show(stats):
  row_names = stats.keys()
  ps = [pr.get_stats(include_mrr5=True) for pr in stats.values()]
  column_names = [ '&', 'hit@1', '&', 'hit@5', '&', 'hit@10', '&', 'MRR@10', '&', 'Processed sentences', '\\\\' ]
  dfdata = np.row_stack([('&', f'{x[0]:.6f}', '&', f'{x[1]:.6f}', '&', f'{x[2]:.6f}', '&', f'{x[5]:.6f}', '&', f'{x[6]}', '\\\\') for x in ps])
  pd.set_option('display.max_rows', 1000)
  pd.set_option('display.min_rows', 1000)
  text_stats_df = pd.DataFrame(dfdata, columns=column_names, index=row_names)
  text_stats_df.index = row_names
  print(text_stats_df)
#stats_show({"W2V TR combined": res, "W2V TR combined2": res2})

#store_stats(res)
#store_stats(res2)

w2v_tr_combined_array = dict()
w2v_tr_combined = None
w2v_tr_combined_score = -1
w2v_tr_combined_stats = dict()

w2v_tr_combined_data = []
w2v_tr_combined_opts = None
w2v_tr_opts = dict()
w2v_tr_id = dict()

# A table of results used to form set of models to try, currently 2 entries.
# Item #1 is optimized using optimization process outlayed in the doc.
resprev = """
1      0.050000        21   0     150          200      2          False  0.010000 4  1.0
2          0.1         21   0     150          200      5           True  0.01  4     0.5
""".split()
resprev = [
    (float(resprev[i + 10]),
     [float(resprev[i + 1]), int(resprev[i + 2]),
      int(resprev[i + 3]), int(resprev[i + 4]), int(resprev[i + 5]),
      int(resprev[i + 6]), bool(resprev[i + 7]), float(resprev[i + 8]), int(resprev[i + 9])],
     int(resprev[i])) for i in range(0,len(resprev),11)]
resprev.sort(key=lambda element: (-element[0], element[2], element[1]))
for test in resprev:
  ns_exponent, negative, sg, epochs, vector_size, window, shrink_window, sample, min_count = test[1]
  t=(ns_exponent, negative, sg, epochs, vector_size, window, shrink_window, sample, min_count)
  if t in w2v_tr_opts:
    continue
  opts = {'ns_exponent': ns_exponent, 'negative': negative, 'sg': sg,
          'vector_size': vector_size, 'window': window,
          'sample': sample,
          'shrink_windows': (shrink_window == 1), 'compute_loss': True, 'epochs': epochs}
  w2v_tr_opts[t] = opts
  w2v_tr_id[t] = f"W2V TR nse={ns_exponent}-neg={negative}-{'sg' if sg else 'cbow'}-epochs={epochs}-vs={vector_size}-w={window}-{'shrink' if shrink_window == 1 else 'static'}-sample={sample}-min={min_count}"
  if len(w2v_tr_opts) >= w2v_tr_combined_variants:
    break

res = None
def get_stats(pr):
  global res
  values = pr.get_stats()
  res = pr
  return values[4]

for t, opts in tqdm_notebook(w2v_tr_opts.items(), desc="Variants"):
  id = w2v_tr_id[t]
  short_id = "W2V-TR-combined-2.01"
  w2v_tr_combined_array[t] = get_gensim_gen(marete_train_sentences_in, token_seq_func = token_seq_out,
                                            silent=True, id=short_id, **opts, # Takes around 2 minutes (testing) + 20 minutes training on the first run
                                            test=lambda model: get_stats(test_w2v(short_id, model, marete_test_sentences_in, seq=token_seq_out, silent=True)))
  #w2v_tr_combined_array[t] = get_gensim_gen(aecorpus.aes_1, token_seq_func = token_seq_tr,
  #                                          silent=True, id="W2V-AES-TR-combined-1.5", noload=True,
  #                                          nosave = True, **opts) # Takes around 2 minutes
  #res = test_w2v(id, w2v_tr_combined_array[t], aecorpus.aes_t, seq=token_seq_tr, silent=True)
  mrr = res.get_stats()[4]
  w2v_tr_combined_stats[id] = res
  # Keep track of best (incase resprev has multiple variants)
  if mrr > w2v_tr_combined_score:
    w2v_tr_combined_score = mrr
    w2v_tr_combined = w2v_tr_combined_array[t]
    w2v_tr_combined_opts = opts
  w2v_tr_combined_data.append([*t,mrr])
pd.set_option('display.width', 1000)
stats_show(w2v_tr_combined_stats)
store_stats(res)
print(w2v_tr_combined_opts)
print(w2v_tr_combined_score)


Using 101466 training sentences and 25377 test sentences. 0 filtered out.


Variants:   0%|          | 0/1 [00:00<?, ?it/s]

                                                    &     hit@1  &     hit@5  &    hit@10  &    MRR@10  & Processed sentences  \\
W2V TR nse=0.05-neg=21-cbow-epochs=150-vs=200-w...  &  0.118336  &  0.328408  &  0.434527  &  0.206749  &               25377  \\
{'ns_exponent': 0.05, 'negative': 21, 'sg': 0, 'vector_size': 200, 'window': 2, 'sample': 0.01, 'shrink_windows': True, 'compute_loss': True, 'epochs': 150}
0.20674875262001402


In [ ]:
# @title Vocabulary viewer. Click `Show code` in the code cell. { display-mode: "form" }
# @markdown Examine vocabularies interactively

@interact(model=[("Word2Vec / RTC", (w2v_rtc, "transliteration")),
                 ("Word2Vec / AES", (w2v, "token", "text")),
                 ("Word2Vec p0 / AES", (w2vp0, "token", "text")),
                 ("Word2Vec p3 / AES", (w2vp3, "token", "text")),
                 ("Word2Vec / AES TR", (w2v_tr, "transliteration")),])
def view_vocabulary(model):
  clear_output()
  fields = list(model)[1:]
  model = model[0]
  if model is not None and model != "":
      vocab = list(model.wv.index_to_key)
      d = dict()
      for f in fields:
        if f == "text":
          d[f.capitalize()] = [token_to_text(token) for token in vocab]
        else:
          d[f.capitalize()] = vocab
      print(f"Vocabulary size: {len(vocab)}")
      pd.set_option('display.min_rows', 25)
      pd.set_option('display.max_rows', 25)
      vocabulary_entries_df = pd.DataFrame(d)
      display(vocabulary_entries_df)



interactive(children=(Dropdown(description='model', options=(('Word2Vec / RTC', (<gensim.models.word2vec.Word2…

In [ ]:
# @title Prediction statistics from training set. Click `Show code` in the code cell. { display-mode: "form" }

def prediction_stats_to_widget(stats_predict):
  predict_html = f"""
  <table border="1">
   <tr>
    <th>Model / #</th>
    <th>Sentence</th>
    <th>Omitted word</th>
    <th>Hit idx</th>
    <th>Guesses</th>
   </tr>
  """
  for stat in stats_predict:
    subtab = tabulate(stat[5], headers=[ "Guess", "Probability", "Similarity with right guess" ], tablefmt='html')
    sentence = " ".join(stat[1])
    if len(sentence) > 32:
      sentence = sentence[:30] + "\u2026"
    predict_html += f"""
     <tr>
      <td>{stat[0]}</td>
      <td>{sentence}</td>
      <td>{stat[2]}</td>
      <td>{stat[3]}</td>
      <td><details><summary>{" ".join(stat[4])}</summary>
           <p>{subtab}</p></details></td>
     </tr>
    """
  predict_html += f"""
  </table>
  """
  return widgets.HTML(
    value=predict_html,
    placeholder='...Processing...',
    description='Predictions:')

print("Some manual trials of predict_words()")
stats_predict=[]
for idx,sentence in enumerate(aecorpus.rtc_t[:5]):
  seq = token_seq_rtc(sentence)
  omitted = seq[3]
  seq[3] = "LACUNA"
  res = predict_words(seq,w2v_rtc,stat_append=stats_predict,stat_id=f"W2V_RTC #{idx+1}", omitted=omitted)
  if train_ft:
    res = predict_words(seq,ft_rtc,stat_append=stats_predict,stat_id=f"FT_RTC #{idx+1}", omitted=omitted)
  res = predict_words(seq,w2v_rtc,stat_append=stats_predict,stat_id=f"Vocab_RTC #{idx+1}", omitted=omitted, algo=-1)
for idx, sentence in enumerate(aecorpus.aes_t[:5]):
  seq = token_seq(sentence)
  omitted = seq[3]
  seq[3] = "LACUNA"
  res = predict_words(seq,w2v,stat_append=stats_predict,stat_id=f"W2V_AES #{idx+1}", omitted=omitted)
  seq = token_seq(sentence,padlen=3)
  omitted = seq[4]
  seq[4] = "LACUNA"
  res = predict_words(seq,w2vp3,stat_append=stats_predict,stat_id=f"W2VAES p3 #{idx+1}",omitted=omitted)
  res = predict_words(seq,w2v,stat_append=stats_predict,stat_id=f"Vocab_AES #{idx+1}", omitted=omitted, algo=-1)

prediction_stats_to_widget(stats_predict)


Some manual trials of predict_words()


HTML(value='\n  <table border="1">\n   <tr>\n    <th>Model / #</th>\n    <th>Sentence</th>\n    <th>Omitted wo…

In [ ]:
# @title Show some samples of `token_seq` and `token_seq_rtc`. Click `Show code` in the code cell. { display-mode: "form" }

for sentence in aecorpus.aesen_1[:1]:
  print('AES', token_seq(sentence)) # Sample sentence
  for t in sentence['tokens']:
    tt = t
    if tt[1] in ge_en:
      tt[1] = ge_en[tt[1]]
    print(tt)
for sentence in aecorpus.rtc_1[:1]:
  print('RTC', token_seq_rtc(sentence)) # Sample sentence
  for t in sentence['tokens']:
    print(t)



AES ['<s0>', '<s1>', '55500', '49520', '10030', '851661', '38560', '851809', '10030', '78870', '851446', '850317', '550021', '10030', '32820', '91900', '86570', '181400', '109000', '21881', '10050', '69590', '107520', '51990', '850787', '27560', '64360', '851446', '169650', '96080', '64830', '64360', '851446', '169650', '859286', '850787', '233', '</s1>', '</s0>']
('⸢bn⸣', '[Negationspartikel]', 'bn', '55500', 'particle')
('⸢wsf⸣', '(einen Eid) brechen', 'wzf', '49520', 'verb')
('=[j]', '[Suffix Pron. sg.1.c.]', '=j', '10030', 'pronoun')
('[pAj]', 'dieser [Dem.Pron. sg.m.]', 'pꜣj', '851661', 'pronoun')
('[anx]', 'Eid', 'ꜥnḫ', '38560', 'substantive')
('[jri.n]', 'machen', 'jri̯', '851809', 'verb')
('=[j]', '[Suffix Pron. sg.1.c.]', '=j', '10030', 'pronoun')
('[n]', '[Dativ: Nutzen]', 'n', '78870', 'preposition')
('[pA]', 'der [Artikel sg.m.]', 'pꜣ', '851446', 'pronoun')
('jtn', 'Aton', 'Jtn', '850317', 'entity_name')
('⸢pAy⸣', '[Poss.artikel sg.m.]', 'pꜣy=', '550021', 'pronoun')
('=j', 

In [ ]:
# @title Genetic algorithm optimizer for Word2Vec parameters. Click `Show code` in the code cell. { display-mode: "form" }
# @markdown Note: To use the optimizer, please see the code.
# @markdown Code *will need to be* manually adapted to set of variables to optimize.
# @markdown Furthermore, genetic algorithm optimization takes hours.
import geneticalgorithm2

#get_gensim_w2v_rtc(corpus, vector_size=200, window=5, min_count=1, epochs=100, hs=0, negative=5, nopad=False, replace_numbers=False, **kwargs):
#test_w2v_rtc(name, w2v, w2v_hs, algo, sentences, nopad=False)
def optimize_w2v_rtc(**kwargs):
  global aecorpus
  w2v_rtc_opt = get_gensim_w2v_rtc(aecorpus.rtc_1, **kwargs)
  prn = test_w2v_rtc("W2V RTC Opt", w2v_rtc_opt, None, 0, aecorpus.rtc_t, silent=True)
  mrr = prn.get_stats()[4]
  return mrr

def optimize_w2v_aes(**kwargs):
  global aecorpus
  w2v_aes_opt = get_gensim_w2v(aecorpus.aes_1, **kwargs)
  padlen=2
  if 'padlen' in kwargs:
    padlen=kwargs['padlen']
  prn = test_w2v("W2V AES Opt", w2v_aes_opt, aecorpus.aes_t, silent=True, padlen=padlen)
  mrr = prn.get_stats()[4]
  return mrr

ga_break=False
def optimize_ga():
  algorithm_param = geneticalgorithm2.AlgorithmParams(
      max_num_iteration = 10,
      population_size = 10,
      mutation_probability = 0.15,
      elit_ratio = 0.02,
      max_iteration_without_improv = 3)
  varbound=np.array([[0.1,1.0],[1,30],[0,1],[50, 300],[2,10],[1,5],[0,1]])
  vartype=('real','int','int','int','int','int','int')
  varnames=['ns_exponent', 'negative', 'sg', 'vector_size', 'window', 'min_count', 'shrink_windows' ]
  static_args=[True, True, 15, 3]
  static_varnames=['compute_loss', 'silent', 'epochs', 'padlen']
  #varbound=np.array([[0.0,0.03],[0.0,0.1]])
  #vartype=np.array([['real'],['real']])
  #The best found solution: for padlen=3
  #[  0.26927101   5.           1.         247.           2.
  #  2.           0.        ]
  # like: {'ns_exponent': 0.20375056149833976, 'negative': 15, 'sg': 0, 'vector_size': 219, 'window': 5, 'min_count': 1, 'shrink_windows': False, 'compute_loss': True, 'silent': True, 'epochs': 15, 'padlen': 3, 'result': 0.12751039243058787}
  def f(x):
    global ga_break
    if ga_break:
      raise Exception("Breaking")
    y = [x[0], int(x[1]), int(x[2]), int(x[3]), int(x[4]), int(x[5]), bool(x[6]) ]
    kwargs=dict(zip(varnames+static_varnames,y+static_args))
    res = optimize_w2v_rtc(**kwargs)
    kwargs['result'] = res
    print(kwargs)
    return 1 - res

  def f2(x):
     # Best solution found: [0.02604439 0.0682137 ]
    global ga_break
    if ga_break:
      raise Exception("Breaking")
    kwargs = {'ns_exponent': 0.3400841723332061, 'negative': 23, 'sg': 1, 'vector_size': 196, 'window': 2, 'min_count': 2, 'shrink_windows': False, 'compute_loss': True, 'silent': True, 'epochs': 10}
    kwargs['alpha'] = x[0]
    kwargs['min_alpha'] = x[1]*x[0]
    print(kwargs)
    res = optimize_w2v_rtc(**kwargs)
    kwargs['result'] = res
    print(res)
    return 1 - res

  def f3(x):
    global ga_break
    if ga_break:
      raise Exception("Breaking")
    y = [x[0], int(x[1]), int(x[2]), int(x[3]), int(x[4]), int(x[5]), bool(x[6]) ]
    kwargs=dict(zip(varnames+static_varnames,y+static_args))
    res = optimize_w2v_aes(**kwargs)
    kwargs['result'] = res
    print(kwargs)
    return 1 - res

# [3.75760378e-01 1.90000000e+01 0.00000000e+00 2.01000000e+02
# 5.00000000e+00 4.00000000e+00 1.00000000e+00 8.57477137e-03
# 2.32058522e-03]

# {'ns_exponent': 0.3400841723332061, 'negative': 23, 'sg': 1, 'vector_size': 196, 'window': 2, 'min_count': 2, 'shrink_windows': False, 'compute_loss': True, 'silent': True, 'epochs': 10, 'result': 0.24762647483235717}

  #               ns_exponent negative  sg
  # Previously determined best values:
  # 'ns_exponent': 0.21856359,
  # 'negative': 28,
  # 'sg': 1,
  model=ga2(function=f3,dimension=len(varbound),variable_type=vartype,
           variable_boundaries=varbound, function_timeout=400,
           algorithm_parameters=algorithm_param)
  return model

# Processing takes tens of minutes if you click button.
ga_layout = widgets.Layout(width='auto', height='40px') #set width and height
ga_button = widgets.Button(
    description="Genetic Algorithm Optimize Word2Vec AES", layout=ga_layout,
    flex_flow='column', align_items='stretch')
ga_button_break = widgets.Button(
    description="Break Processing", layout=ga_layout,
    flex_flow='column', align_items='stretch')
ga_output = widgets.Output()

display(ga_button, ga_button_break, ga_output)

# Create model storage file
ga_filename = [ "aes_ga_p3b_lastgen", ".npz" ]
ga_run_args = {}
ga_idx = 0
while os.path.exists("models/" + ga_filename[0] + str(ga_idx) + ga_filename[1]):
  ga_run_args['start_generation'] = "models/" + ga_filename[0] + str(ga_idx) + ga_filename[1]
  ga_idx += 1

ga_run_args['save_last_generation_as'] = "models/" + ga_filename[0] + str(ga_idx) + ga_filename[1]

def ga_on_button_clicked(b):
    global ga_output
    with ga_output:
        global ga_break
        ga_break=False
        import threading
        import traceback
        def optimizer():
          print("Started optimizing")
          try:
            optimize_ga().run(**ga_run_args)
          except:
            traceback.print_exc()
          print("Optimizer finished")
        #thread = threading.Thread(target=optimizer)
        #thread.start()
        optimizer()

def ga_break_button_clicked(b):
    global ga_output
    with ga_output:
        print("*BREAK*")
        global ga_break
        ga_break=True

ga_button.on_click(ga_on_button_clicked)
ga_button_break.on_click(ga_break_button_clicked)
#with ga_output:
#  population_initializer = geneticalgorithm2.Population_initializer(
#      select_best_of = 4,
#      local_optimization_step = 'never', local_optimizer = None)
#  optimize_ga().run(**ga_run_args, studEA=True,
#                    population_initializer = population_initializer)


Button(description='Genetic Algorithm Optimize Word2Vec AES', layout=Layout(height='40px', width='auto'), styl…

Button(description='Break Processing', layout=Layout(height='40px', width='auto'), style=ButtonStyle())

Output()

# Ngrams

In [ ]:
# @title Provide Ngrams. Click `Show code` to view details.  { display-mode: "form" }
# @markdown The Ngrams are organized in dict to predict middle word.
# @markdown
# @markdown These take a few minutes to test.

# `from_ngrams` copied from BLEU exercise:
from collections import deque

def form_ngrams(tokens, n):
    """Forms all ngrams from tokens

    Arguments
    ---------
    tokens : iterable
        Tokens to form n-grams from
    n : int
        The length of ngrams to form

    Yields
    ------
    tuple
        Yields each ngram as a tuple of tokens.
    """
    window = deque(maxlen=n)
    for i, token in enumerate(tokens, start=1):
        window.append(token)
        if i >= n:
            yield tuple(window)

# Ngram model
# This class has been carefully defined to be compatible with Word2Vec's
# models from perspective of (the few used) API functions.
# The class needs to provide __init__ and predict_output_word.
# For predict_output_word, ngrams are split and placed in dictionary
# left-most ngrams, right-most ngrams. In case n is odd, right side is
# larger (the context word to be found is on the right side).
#
# This can be used with test_w2v and underlying predict_words with algo=0
class NgramModel:
  def __init__(self, tokens, seq=token_seq, n=3):
    self.n = n
    c = Counter()
    for s in tokens:
      for ng in form_ngrams(seq(s), n):
        c[ng] += 1
    # middle counter: dictionary of (left_side,) => ((right_side,), counter)
    self.mc = dict()
    self.right = (n + 1) // 2 # Right side context length
    self.left = n - self.right # Left side context length
    for k in c.most_common():
      lt = tuple(k[0][:self.left])
      rt = tuple(k[0][self.left:])
      if lt not in self.mc:
        self.mc[lt] = []
      self.mc[lt].append((rt, k[1]))
    self.seq = seq
    self.total = c.total()
    top_ngram = c.most_common(1)
    self.top_ngram_count = top_ngram[0][1] if len(top_ngram) >= 1 else 1
    # The actual ngram statistic c is not stored. For other uses of ngrams it could be useful.
    # Uncomment if you need it.
    # self.c = c

  def predict_output_word(self, seq, topn, verbose=False):
    # Function may be called with varying amounts of context.
    # We need to use exactly n - 1 words of context.
    # Find a context with size n - 1
    if len(seq) < self.n - 1:
      return []
    while len(seq) >= self.n + 1:
      seq = seq[1:len(seq)-1] # Keep middle of context
    while len(seq) >= self.n:
      seq = seq[0:len(seq)-1] # Dismiss last
    left = tuple(seq[:self.left])
    right = tuple(seq[self.left:])

    # Search for results: match left side with dictionary lookup, ...
    r = []
    for n in self.mc.get(left, []):
      # ... and right side with equality
      if n[0][1:] == right:
        if is_pad(n[0][0]):
          continue
        # Add results
        r.append((n[0][0], n[1]/self.top_ngram_count))
        # If there are extraneous results, throw
        if len(r) >= topn:
          break
    if verbose:
      print(r)
    return r

def store_stats(pr):
  stats[pr.get_name()] = pr
  values = pr.get_stats()
  return values[4]

build_ngrams = True
build_ngrams_aes = build_ngrams
build_ngrams_aes_tr = build_ngrams
build_ngrams_rtc = build_ngrams


for x in tqdm_notebook(range(3), desc="Train"):
  if x == 0 and build_ngrams_aes:
    ngram_aes = [None]*10
    for n in tqdm_notebook(range(1, 10), desc="n", leave=False):
      ngram_aes[n] = NgramModel(aecorpus.aes_1, n=n)
      pr = test_w2v(f'{n}-grams AES', ngram_aes[n], sentences=aecorpus.aes_t, silent=True)
      store_stats(pr)

  if x==1 and build_ngrams_aes_tr:
    ngram_aes_tr = [None]*10
    for n in tqdm_notebook(range(1, 10), desc="n", leave=False):
      ngram_aes_tr[n] = NgramModel(aecorpus.aes_1, seq=token_seq_tr, n=n)
      pr = test_w2v(f'{n}-grams AES TR', ngram_aes_tr[n], sentences=aecorpus.aes_t, seq=token_seq_tr, silent=True)
      store_stats(pr)

  if x==2 and build_ngrams_rtc:
    ngram_rtc = [None]*10
    for n in tqdm_notebook(range(1, 10), desc="n", leave=False):
      ngram_rtc[n] = NgramModel(aecorpus.rtc_1, seq=token_seq_rtc, n=n)
      pr = test_w2v(f'{n}-grams RTC', ngram_rtc[n], sentences=aecorpus.rtc_t, seq=token_seq_rtc, silent=True)
      store_stats(pr)



Train:   0%|          | 0/3 [00:00<?, ?it/s]

n:   0%|          | 0/9 [00:00<?, ?it/s]

n:   0%|          | 0/9 [00:00<?, ?it/s]

n:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
# @title Provide Ngrams results Aggregator. Click `Show code` to view details.  { display-mode: "form" }

# @markdown bigrams are not very accurate, 7-grams are rarely found. Combine
# @markdown these into new aggregator, that takes results from highest ngrams
# @markdown available.
# @markdown
# @markdown These take a few minutes to test.

class NgramAggregate:
  def __init__(self, ngrams):
    self.ngrams = tuple(reversed(ngrams))
  def predict_output_word(self, seq, topn, verbose=False):
    results = []
    filter_results = set()
    for ng in self.ngrams:
      if verbose == False:
        new_results = ng.predict_output_word(seq, topn)
      else:
        new_results = ng.predict_output_word(seq, topn, verbose)
      # Only append unique results.
      for a in new_results:
        if a[0] not in filter_results:
          results.append(a) # Ignore scoring
          filter_results.add(a[0])
          if len(results) >= topn:
            break
      if len(results) >= topn:
        break
    # Replace scoring (to remove inconsistencies between different n-grams in the aggregate)
    for idx in range(len(results)):
      results[idx] = (results[idx][0], 1 / (idx + 1))
    return results

# Build combined NgramAggregate

for x in tqdm_notebook(range(3), desc="Test Model", leave=False):
  if x == 0 and build_ngrams_aes:
    nga = NgramAggregate(ngram_aes[1:])
    pr = test_w2v('x-ngrams AES', nga, sentences=aecorpus.aes_t, silent=True)
    store_stats(pr)
  if x == 1 and build_ngrams_aes_tr:
    nga_tr = NgramAggregate(ngram_aes_tr[1:])
    pr = test_w2v('x-ngrams AES TR', nga_tr, sentences=aecorpus.aes_t, seq=token_seq_tr, silent=True)
    store_stats(pr)
  if x == 2 and build_ngrams_rtc:
    nga_rtc = NgramAggregate(ngram_rtc[1:])
    pr = test_w2v('x-ngrams RTC', nga_rtc, sentences=aecorpus.rtc_t, seq=token_seq_rtc, silent=True)
    store_stats(pr)



Test Model:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Train and test ngram models (for combined)
ngram_tr_combined = [None]*10
for n in tqdm_notebook(range(1, 10), desc="Train", leave=False):
  ngram_tr_combined[n] = NgramModel(marete_train_sentences_in, seq=token_seq_out, n=n)
  pr = test_w2v(f'{n}-grams TR combined', ngram_tr_combined[n], sentences=marete_test_sentences_in, seq=token_seq_out, silent=True)
  store_stats(pr)


Train:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
# @markdown optional training for 2,9-grams TR combined
train_nga2_9_tr=False # @param {type:"boolean"}

# Construct and test aggregation of ngram models (for combined)
if train_nga2_9_tr:
  nga_tr_combined_old = NgramAggregate(ngram_tr_combined[2:])
  pr = test_w2v('2,9-grams TR combined', nga_tr_combined_old,
                sentences=tqdm_notebook(marete_test_sentences_in, desc="Sentence", leave=False), seq=token_seq_out, silent=True)
  store_stats(pr)


In [ ]:
# Construct and test aggregation of ngram models (for combined)
# Takes 5 minutes
nga_tr_combined = NgramAggregate(ngram_tr_combined[1:])
pr = test_w2v('1,9-grams TR combined', nga_tr_combined,
              sentences=tqdm_notebook(marete_test_sentences_in, desc="Sentence", leave=False), seq=token_seq_out, silent=True)
store_stats(pr)

Sentence:   0%|          | 0/25377 [00:00<?, ?it/s]

0.46213278365424015

In [ ]:
# Consider storage requirements of ngram models. (Currently not saved.)

if False: # Set to true if you wish to see how much space storing data structures uses
  import zlib
  import json
  import pickle

  sum1 = 0
  sum2 = 0
  for n in range(1,10):
    mc = ngram_tr_combined[n].mc
    mcl = []
    for key, value in list(mc.items()):
      left = list(key)
      right = [[list(l[0]), l[1]] for l in value]
      mcl.append([left, right])
    jsobj = json.dumps(mcl).encode('utf-8')
    jsobjgz = zlib.compress(jsobj, level=9)
    print(n, len(mc), len(jsobj), len(jsobjgz), len(pickle.dumps(ngram_tr_combined[n])))
    sum1 += len(jsobj)
    sum2 += len(jsobjgz)
  print(f"Total: plaintext_json={sum1} compressed_json={sum2}")
  import time
  start = time.time()
  combined_pickle = pickle.dumps(nga_tr_combined, protocol=pickle.HIGHEST_PROTOCOL)
  picklegz = zlib.compress(combined_pickle, level=9)
  end = time.time()
  print(f"combined pickle + compress: len={len(combined_pickle)}, lengz={len(picklegz)}, time={end - start}")
  start = time.time()
  pickled = zlib.decompress(picklegz)
  end = time.time()
  print(f"decompress: time={end - start}")
  start = time.time()
  res = pickle.loads(pickled)
  end = time.time()
  print(f"unpickle: time={end - start}")




In [ ]:
# Construct and test another aggregation (for combined)
# This one uses w2v_tr_combined as fallback, so that predictions are possible
# even when no matched

# @markdown optional training for 2,9-grams TR combined with Word2Vec fallback
train_nga2_9_w2v_tr=False # @param {type:"boolean"}

if train_nga2_9_w2v_tr:
  nga_tr_combined_alt = NgramAggregate([w2v_tr_combined] + ngram_tr_combined[2:])
  pr = test_w2v('2,9-ngrams TR combined+w2v', nga_tr_combined_alt,
                sentences=tqdm_notebook(marete_test_sentences_in, desc="Sentence", leave=False), seq=token_seq_out, silent=True)
  store_stats(pr)


In [ ]:
# @title Train POS models and perform testing. Click `Show code` in the code cell. { display-mode: "form" }
# @markdown The trained Word2Vec POS models are stored on Google drive.
# @markdown Also train POS model using ngrams.
# @markdown
# @markdown When all models are retrained expect execution of the cell to take
# @markdown around 5 minutes.

# @markdown This is optional step
train_pos_models=False # @param {type:"boolean"}


# Helper function: get sequence of POS from lemmatized AES sentence
def token_seq_pos(sentence, replace_numbers=False, padlen=2, nopad=False):
    # Note: replace numbers is no-op here. replacement has already been performed
    token_seq = [token[4] for token in sentence['tokens']] # With lemma id
    if nopad:
      return token_seq
    return start_pad(padlen) + token_seq + end_pad(padlen)

pos_stats = dict()
def pos_store_stats(pr):
  pos_stats[pr.get_name()] = pr
  values = pr.get_stats()
  return values[4]
for x in tqdm_notebook(range(4), desc="Model", leave=False):
  if x == 0 and train_pos_models:
    padlen = 2
    w2vpos = get_gensim_gen(aecorpus.aes_1, token_seq_func = token_seq_pos,
                           silent=True, padlen=padlen, id="W2V-AES-pos-sg-150",
                           test=lambda model: pos_store_stats(test_w2v("W2V-pos AES", model, aecorpus.aes_t, padlen=padlen, silent=True, seq=token_seq_pos)),
                           **optimized_w2v_sg_opts) # Takes around 2 minutes
  if x == 0 and train_pos_models:
    opts_known_good_150 = {'ns_exponent': 0.21821168418246245, 'negative': 21, 'sg': 1,
                   'vector_size': 227, 'window': 2, #'min_count': 4,
                   'shrink_windows': False, 'compute_loss': True, 'epochs': 150}
    w2vpos2 = get_gensim_gen(aecorpus.aes_1, token_seq_func = token_seq_pos,
                           silent=True, padlen=padlen, id="W2V-AES-pos-good-150",
                           test=lambda model: pos_store_stats(test_w2v("W2V-pos AES (alt)", model, aecorpus.aes_t, padlen=padlen, silent=True, seq=token_seq_pos)),
                           **opts_known_good_150) # Takes around 2 minutes
  if x == 1 and train_pos_models:
    # Provide alternative without hand-tuned options.
    padlen = 2
    opts = { 'sg': 1, 'compute_loss': True, 'epochs': 15 }
    w2vpos_alt = get_gensim_gen(aecorpus.aes_1, token_seq_func = token_seq_pos,
                           silent=True, padlen=padlen, id="W2V-AES-pos_alt",
                           test=lambda model: pos_store_stats(test_w2v("W2V-pos-alt AES", model, aecorpus.aes_t, padlen=padlen, silent=True, seq=token_seq_pos)),
                           **opts) # Takes around 2 minutes
  if x == 2 and train_pos_models:
    ngram_aes_pos = [None]*10
    for n in tqdm_notebook(range(1, 10), desc="Ngrams", leave=False):
      ngram_aes_pos[n] = NgramModel(aecorpus.aes_1, seq=token_seq_pos, n=n)
      pr = test_w2v(f'{n}-grams AES POS', ngram_aes_pos[n], sentences=aecorpus.aes_t, silent=True, seq=token_seq_pos)
      pos_store_stats(pr)
  if x == 3 and train_pos_models:
    nga_pos = NgramAggregate(ngram_aes_pos[1:])
    pr = test_w2v('x-ngrams AES POS', nga_pos, sentences=aecorpus.aes_t, silent=True, seq=token_seq_pos)
    pos_store_stats(pr)

def pos_stats_update():
  row_names = pos_stats.keys()
  column_names = list(pos_stats.values())[0].get_heading()
  ps = [pr.get_stats() for pr in pos_stats.values()]
  dfdata = np.row_stack([(f'{x[0]:.4f}', f'{x[1]:.4f}', f'{x[2]:.4f}', f'{x[3]:.4f}', f'{x[4]:.4f}', f'{x[5]}', f'{x[6]}', f'{x[7]:.4f}') for x in ps])
  pd.set_option('display.max_rows', 13)
  pd.set_option('display.min_rows', 13)
  text_stats_df = pd.DataFrame(dfdata, columns=column_names, index=row_names)
  text_stats_df.index = row_names
  display(text_stats_df)

def pos_stats_latex():
  row_names = pos_stats.keys()
  column_names = [ '&', 'hit@1', '&', 'hit@5', '&', 'hit@10', '&', 'MRR@10', '&', 'Processed sentences', '\\\\' ]

  #list(stats.values())[0].get_heading(include_mrr5=True)
  ps = [pr.get_stats(include_mrr5=True) for pr in pos_stats.values()]
  dfdata = np.row_stack([('&', f'{x[0]:.6f}', '&', f'{x[1]:.6f}', '&', f'{x[2]:.6f}', '&', f'{x[5]:.6f}', '&', f'{x[6]}', '\\\\') for x in ps])
  pd.set_option('display.max_rows', 90)
  pd.set_option('display.min_rows', 90)
  text_stats_df = pd.DataFrame(dfdata, columns=column_names, index=row_names)
  text_stats_df.index = row_names
  pd.set_option('display.width', 1000)
  print(text_stats_df)

if train_pos_models:
  print("Trained POS Models are now available: " + (", ".join(pos_stats.keys())))
  pos_stats_update()
  pos_stats_latex()

Model:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# @title Provide POS enhanced models. Click `Show code` to view details.  { display-mode: "form" }

# @markdown These combine a word prediction model with pos prediction model to enhance accuracy.
# @markdown
# @markdown These take a few minutes to test.


# @markdown This is optional step
train_experimental_pos_models=False # @param {type:"boolean"}

# Sequence of lemma and pos tuples
def token_seq_lemma_pos(sentence, replace_numbers=False, padlen=2, nopad=False):
    # Note: replace numbers is no-op here. replacement has already been performed
    token_seq = [(token[3], token[4]) for token in sentence['tokens']] # With lemma id
    if nopad:
      return token_seq
    spad = start_pad(padlen)
    epad = end_pad(padlen)
    spad = [(t, t) for t in spad]
    epad = [(t, t) for t in epad]
    return spad + token_seq + epad

# Sequence of pos from lemma sequence
def token_seq_pos_from_lemma_seq(sq):
  s = []
  for item in sq:
    s.append(pos_for_token[item])
  return s

# Create pos_for_token mapping
pos_for_token = dict()
count=0
for s in aecorpus.aes:
  # + aecorpus.aes_1 + aecorpus.aes_t + aecorpus.aes_g:
  count +=1
  seq_lp = token_seq_lemma_pos(s)
  for lp in seq_lp:
    pos_for_token[lp[0]] = lp[1]
# print(f"{count} sentences processed")

# Model that uses POS to rebalance predictions
class ModelWithPos:
  def __init__(self, model, pos_model):
    self.model = model
    self.pos_model = pos_model
  def predict_output_word(self, token_seq, topn, verbose=False):
      global print_count
      seq_pos = token_seq_pos_from_lemma_seq(token_seq)
      if verbose == False:
        results = self.model.predict_output_word(token_seq, topn * 2)
      else:
        results = self.model.predict_output_word(token_seq, topn * 2, verbose)
      results_pos = self.pos_model.predict_output_word(seq_pos, 10, verbose)
      cnt = Counter()
      for idx in range(len(results)):
        pred = results[idx][0]
        pos = pos_for_token.get(pred, "_unknown_")
        value = results[idx][1]
        pos_value = 0.001 # Minimum value (when POS not in top 10)
        for p in results_pos:
          if p[0] == pos:
            pos_value = max(p[1], pos_value)
        cnt[pred] += value * pos_value
      results_new = cnt.most_common(topn)
      return results_new

# Select POS model
if train_experimental_pos_models:
  #pos_model = ngram_aes_pos[5]
  pos_model = nga_pos

for x in tqdm_notebook(range(2), desc="Model", leave=False):
  if x == 0 and train_experimental_pos_models:
    w2v2 = ModelWithPos(w2v, pos_model)
    pr = test_w2v('w2v AES w/pos', w2v2, sentences=aecorpus.aes_t, silent=True)
    store_stats(pr)
  if x == 1 and train_experimental_pos_models:
    nga2 = ModelWithPos(nga, pos_model)
    pr = test_w2v('x-ngrams AES w/pos', nga2, sentences=aecorpus.aes_t, silent=True)
    store_stats(pr)

Model:   0%|          | 0/2 [00:00<?, ?it/s]

# Results

In [ ]:
# @title Display statistics collected from model testing. Click `Show code` in the code cell. { display-mode: "form" }

def stats_update():
  row_names = stats.keys()
  column_names = list(stats.values())[0].get_heading(include_mrr5=True)
  ps = [pr.get_stats(include_mrr5=True) for pr in stats.values()]
  dfdata = np.row_stack([(f'{x[0]:.4f}', f'{x[1]:.4f}', f'{x[2]:.4f}', f'{x[3]:.4f}', f'{x[4]:.4f}' , f'{x[5]:.4f}', f'{x[6]}', f'{x[7]}', f'{x[8]:.4f}') for x in ps])
  pd.set_option('display.max_rows', 40)
  pd.set_option('display.min_rows', 40)
  text_stats_df = pd.DataFrame(dfdata, columns=column_names, index=row_names)
  text_stats_df.index = row_names
  display(text_stats_df)

stats_button = widgets.Button(description="Refresh Stats")
stats_output = widgets.Output()
with stats_output:
  stats_update()

display(stats_button, stats_output)

def on_button_clicked(stats_button):
    with stats_output:
        clear_output(wait = False)
        stats_update()


stats_button.on_click(on_button_clicked)


Button(description='Refresh Stats', style=ButtonStyle())

Output()

In [ ]:
# Provide results as text only (for cut&paste)
# TBD: This cell may be removed after we're done with the report.

def stats_update_text():
  row_names = stats.keys()
  column_names = list(stats.values())[0].get_heading(include_mrr5=True)
  ps = [pr.get_stats(include_mrr5=True) for pr in stats.values()]
  dfdata = np.row_stack([(f'{x[0]:.4f}', f'{x[1]:.4f}', f'{x[2]:.4f}', f'{x[3]:.4f}', f'{x[4]:.4f}' , f'{x[5]:.4f}', f'{x[6]}', f'{x[7]}', f'{x[8]:.4f}') for x in ps])
  pd.set_option('display.max_rows', 90)
  pd.set_option('display.min_rows', 90)
  text_stats_df = pd.DataFrame(dfdata, columns=column_names, index=row_names)
  text_stats_df.index = row_names
  pd.set_option('display.width', 1000)
  print(text_stats_df)
def stats_update_latex():
  row_names = stats.keys()
  column_names = [ '&', 'hit@1', '&', 'hit@5', '&', 'hit@10', '&', 'MRR@10', '&', 'Processed sentences', '\\\\' ]

  #list(stats.values())[0].get_heading(include_mrr5=True)
  ps = [pr.get_stats(include_mrr5=True) for pr in stats.values()]
  dfdata = np.row_stack([('&', f'{x[0]:.6f}', '&', f'{x[1]:.6f}', '&', f'{x[2]:.6f}', '&', f'{x[5]:.6f}', '&', f'{x[6]}', '\\\\') for x in ps])
  pd.set_option('display.max_rows', 90)
  pd.set_option('display.min_rows', 90)
  text_stats_df = pd.DataFrame(dfdata, columns=column_names, index=row_names)
  text_stats_df.index = row_names
  pd.set_option('display.width', 1000)
  print(text_stats_df)
def stats_update_latex2():
  row_names = stats.keys()
  column_names = [ '&', 'hit@1', '&', 'hit@5', '&', 'hit@10', '&', 'MRR@10', '&', 'Processed sentences', '\\\\' ]

  #list(stats.values())[0].get_heading(include_mrr5=True)
  ps = [pr.get_stats(include_mrr5=True) for pr in stats.values()]
  #          self.c[1] / n,
  #        accu5 / n,
  #        accu10 / n,
  #        self.c[-1] / n,
  #        accu_ / n,
  #        accu / n,
  #        n,
  #        self.fails,
  #        ((self.end - self.start) * 1000) / n
  for stat in ps:
    n = stat[6]
    f = stat[7]
    nf = n - f
    stat[0] = stat[0] * n / nf
    stat[1] = stat[1] * n / nf
    stat[2] = stat[2] * n / nf
    stat[3] = stat[3] * n / nf
    stat[4] = stat[4] * n / nf
    stat[5] = stat[5] * n / nf
    stat[6] = f"{nf}/{n} ({round(100*nf/n)}\\%)"
  dfdata = np.row_stack([('&', f'{x[0]:.6f}', '&', f'{x[1]:.6f}', '&', f'{x[2]:.6f}', '&', f'{x[5]:.6f}', '&', f'{x[6]}', '\\\\') for x in ps])
  pd.set_option('display.max_rows', 90)
  pd.set_option('display.min_rows', 90)
  text_stats_df = pd.DataFrame(dfdata, columns=column_names, index=row_names)
  text_stats_df.index = row_names
  pd.set_option('display.width', 1000)
  print(text_stats_df)
stats_update_text()
print()
stats_update_latex()
print("Excl failing:")
stats_update_latex2()



                          hit   hit@5  hit@10  missed   MRR@5  MRR@10      N untestable  ms/test
W2V AES                0.1625  0.3663  0.4527  0.5471  0.2359  0.2475  15419          3   1.8530
W2V-p0 AES             0.0706  0.2250  0.3007  0.2832  0.1238  0.1339  15419       6417   1.1674
W2V-p1 AES             0.1357  0.3329  0.4306  0.5692  0.2061  0.2192  15419          3   1.1783
W2V-p3 AES             0.0998  0.2710  0.3771  0.6227  0.1595  0.1737  15419          3   1.9523
W2V RTC                0.1629  0.3967  0.4930  0.4895  0.2476  0.2606   1142         20   0.8962
W2V AES TR             0.1230  0.3079  0.3843  0.6157  0.1896  0.1998  15419          0   1.2435
W2V-TR-combined-2.01   0.1183  0.3284  0.4345  0.5655  0.1925  0.2067  25377          0   4.2598
1-grams AES            0.0424  0.1672  0.2565  0.7433  0.0848  0.0968  15419          3   0.1730
2-grams AES            0.1558  0.3296  0.4023  0.5845  0.2199  0.2296  15419        204   0.1441
3-grams AES            0.2382 

# Gaps

In [ ]:
# @title Use `W2V AES` and `AES TR` model against known gaps. Click `Show code` in the code cell. { display-mode: "form" }

# @markdown Select language to use for translations.
prefer_en = True #@param {type:"boolean"}

lemmaform_map = dict()
def to_lemma_id(transliteration):
  if transliteration in lemmaform_map:
    return lemmaform_map[transliteration].most_common(1)[0][0]
  if transliteration in written_form_to_mdc:
    transliteration = written_form_to_mdc[transliteration]
    if transliteration in lemmaform_map:
      return lemmaform_map[transliteration].most_common(1)[0][0]
  return None

for sentence in aecorpus.aes:
  for token in sentence['tokens']:
    if token[2] not in lemmaform_map:
      lemmaform_map[token[2]] = Counter()
    lemmaform_map[token[2]] += Counter({token[3]: 1})

slw = []
cn=0
c=0
with open("all_gaps_id.txt") as gaps:
  lacunae = re.compile(r'LACUNAE')
  lacunae_word = re.compile(r'<LACUNAE_WORD>')
  for l in gaps.readlines():
    id,tr = l.split(" ", 1)
    id = id[1:-1]
    if id not in aes_sentence_dictionary:
      if c < 10:
        print(f"{id} not found")
      cn += 1
      continue
    sd = aes_sentence_dictionary[id]
    tokens = []
    if 'tokens' in sd:
      if '850830' in sd['token_list']:
        continue # Missing item is a name => generally not possible to match with models.
      tokens = " ".join([f"{x[3]}/{x[0]}" for x in sd['tokens']])
      ftokens = [(True if x[0] == '[___]' else False) for x in sd['tokens']]
      token_list = [('LACUNA' if x[0] == '[___]' else x[3]) for x in sd['tokens']]
      if 'LACUNA' not in token_list:
        continue # Could not find LACUNA in tokens.
    if prefer_en == False:
      transl = sd['translation'] if 'translation' in sd else "No translation"
    else:
      transl = sd['en'] if 'en' in sd else (sd['translation'] if 'translation' in sd else "No translation")
    tl = sd['sentence_transliteration'] if 'sentence_transliteration' in sd else ''
#  extra_info[id] = (s['token_list'], s['sentence_transliteration'], trans)
    t = [tr.split(" "), transl, tokens, tl, token_list ]
    res = lacunae.findall(tr)
    if len(res) == 1 and len(lacunae_word.findall(tr)) == 1:
      slw.append(t)
      c += 1
if cn > 0:
  print(f"{c} suitable sentences; {cn} sentences not found")
else:
  print(f"{c} suitable sentences")
def get_predictions(list_of_inputs):
  out = [
    f"""<form>
      """ ]
  for idx,t in enumerate(list_of_inputs):
    tokens = t[0]
    #print(tokens)
    pr = predict_words(start_pad() + tokens + end_pad(),w2v_tr,lacuna='<LACUNAE_WORD>')
    prx = predict_words(start_pad() + t[4] + end_pad(),w2v,lacuna='LACUNA')
    #print(start_pad() + t[4] + end_pad())
    pr2 = Counter()
    for p in prx:
      if not hasattr(p, '__iter__'):
        continue
      if is_pad(p[0]):
        continue
      #print(p)
      lid = p[0]
      lform = None
      transl = None
      if lid is not None:
        #if len(lid) > 1:
        #  not unique
        lid = str(lid)
        if lid in aecorpus.aed:
          lform = aecorpus.aed[lid]['form']
          if 'translations' in aecorpus.aed[lid]:
            transl = aecorpus.aed[lid]['translations']
            d = dict([(k[1], k[0]) for k in transl])
            if prefer_en == False and 'de' in d:
              transl = d['de']
            elif 'en' in d:
              transl = d['en']
            elif 'de' in d:
              transl = d['de']
            else:
              transl = str(d)
        else:
          lform = "(unknown)"
      pr2[(lform, lform, lid, transl)] += p[1]
    for p in pr:
      if is_pad(p[0]):
        continue
      lid = to_lemma_id(p[0])
      lform = None
      transl = None
      if lid is not None:
        #if len(lid) > 1:
        #  not unique
        lid = str(lid)
        if lid in aecorpus.aed:
          lform = aecorpus.aed[lid]['form']
          if 'translations' in aecorpus.aed[lid]:
            transl = aecorpus.aed[lid]['translations']
            d = dict([(k[1], k[0]) for k in transl])
            if 'en' in d:
              transl = d['en']
            elif 'de' in d:
              transl = d['de']
            else:
              transl = str(d)
        else:
          lform = "(unknown)"
      pr2[(p[0], lform, lid, transl)] += p[1]
    # Combine results from both models
    pr2 = pr2.most_common()
    pr2 = [[p[1]] + list(p[0]) for p in pr2]
    #print(pr2)
    if False: # Use details/summary elements (old style)
        subtab = tabulate(pr2, headers=[ "Probability", "Guess", "Lemma Form", "Lemma ID", "Translation" ], tablefmt='html')
        s1 = '<details style="display: inline"><summary style="display: inline">'
        s2 = f'</summary> <p style="display: inline">{subtab}</p></details>'
    elif True:
      # Use datalist and input
      s1 = f'<input size="20" style="width: 300px; -webkit-rtl-ordering: logical;" type=text id=options-input-{idx} list=options-{idx}><!-- ' # value="{pr2[0][1]}"><!--
      s2 = f'--><datalist class="keepDatalistOptions" size="20" id=options-{idx}>'
      for p in pr2:
        s2 += f'<option value="{p[1]}">{p[2]} {p[3]} {p[4]} ({p[0]})'
      s2 += f'</datalist>'
    elif False:
      # Use select/optgroup/option
      js_opts = ''
      # alt behaviour - shows all options:
      size=len(pr2)
      js_opts = f"onfocus='this.size={size*2};' onblur='this.size=1;' onchange='this.size=1; this.blur();'"
      s1 = f'<select style="width: 500px;" type=text {js_opts} value="{pr2[0][1]}"><!-- '
      s2 = f'-->'
      for p in pr2:
        s2 += f'<optgroup label="{p[2]} {p[3]} {p[4]} ({p[0]:0.5f})"><option value="{p[1]}">{p[1]}</optgroup>'

    out.append(
        f"""
           <p>{" ".join(tokens).replace(">", "&extended-gt;").replace("<", s1 + "&lt;").replace("&extended-gt;", "&gt;" + s2)}</p>
        """)
    # optional extra context information
    #out.append("<p>"+(str(t[2]).replace("<", "&lt;").replace(">", "&gt;"))+"</p>")
    #out.append("<p>"+str(t[4])+"</p>")
    #out.append("<p>"+str(t[3])+"</p>")
    out.append("<p>"+str(t[1])+"</p>")
    out.append("<hr>")
  out[-1] = '</form>'
  return "".join(out)
w = widgets.HTML(
    value=get_predictions(slw[:50]),
    placeholder='...Processing...',
    description='Predictions for gaps:')
display(w)

60 suitable sentences


HTML(value='<form>\n      \n           <p>ztp.n sw jtj =f Jmn m <input size="20" style="width: 300px; -webkit-…

In [ ]:

# @title Use `Ngrams aggregator AES` model against known gaps. Click `Show code` in the code cell. { display-mode: "form" }

# @markdown Select language to use for translations.
prefer_en = True #@param {type:"boolean"}

lemmaform_map = dict()
def to_lemma_id(transliteration):
  if transliteration in lemmaform_map:
    return lemmaform_map[transliteration].most_common(1)[0][0]
  if transliteration in written_form_to_mdc:
    transliteration = written_form_to_mdc[transliteration]
    if transliteration in lemmaform_map:
      return lemmaform_map[transliteration].most_common(1)[0][0]
  return None

for sentence in aecorpus.aes:
  for token in sentence['tokens']:
    if token[2] not in lemmaform_map:
      lemmaform_map[token[2]] = Counter()
    lemmaform_map[token[2]] += Counter({token[3]: 1})

slw = []
cn=0
c=0
with open("all_gaps_id.txt") as gaps:
  lacunae = re.compile(r'LACUNAE')
  lacunae_word = re.compile(r'<LACUNAE_WORD>')
  for l in gaps.readlines():
    id,tr = l.split(" ", 1)
    id = id[1:-1]
    if id not in aes_sentence_dictionary:
      if c < 10:
        print(f"{id} not found")
      cn += 1
      continue
    sd = aes_sentence_dictionary[id]
    tokens = []
    if 'tokens' in sd:
      if '850830' in sd['token_list']:
        continue # Missing item is a name => generally not possible to match with models.
      tokens = " ".join([f"{x[3]}/{x[0]}" for x in sd['tokens']])
      ftokens = [(True if x[0] == '[___]' else False) for x in sd['tokens']]
      token_list = [('LACUNA' if x[0] == '[___]' else x[3]) for x in sd['tokens']]
      if 'LACUNA' not in token_list:
        continue # Could not find LACUNA in tokens.
    if prefer_en == False:
      transl = sd['translation'] if 'translation' in sd else "No translation"
    else:
      transl = sd['en'] if 'en' in sd else (sd['translation'] if 'translation' in sd else "No translation")
    tl = sd['sentence_transliteration'] if 'sentence_transliteration' in sd else ''
#  extra_info[id] = (s['token_list'], s['sentence_transliteration'], trans)
    t = [tr.split(" "), transl, tokens, tl, token_list ]
    res = lacunae.findall(tr)
    if len(res) == 1 and len(lacunae_word.findall(tr)) == 1:
      slw.append(t)
      c += 1
if cn > 0:
  print(f"{c} suitable sentences; {cn} sentences not found")
else:
  print(f"{c} suitable sentences")
def get_predictions(list_of_inputs):
  out = [
    f"""<form>
      """ ]
  for idx,t in enumerate(list_of_inputs):
    tokens = t[0]
    #print(tokens)
    prx = predict_words(start_pad() + t[4] + end_pad(),nga,lacuna='LACUNA')
    #print(start_pad() + t[4] + end_pad())
    pr2 = Counter()
    for p in prx:
      if not hasattr(p, '__iter__'):
        continue
      if is_pad(p[0]):
        continue
      #print(p)
      lid = p[0]
      lform = None
      transl = None
      if lid is not None:
        #if len(lid) > 1:
        #  not unique
        lid = str(lid)
        if lid in aecorpus.aed:
          lform = aecorpus.aed[lid]['form']
          if 'translations' in aecorpus.aed[lid]:
            transl = aecorpus.aed[lid]['translations']
            d = dict([(k[1], k[0]) for k in transl])
            if prefer_en == False and 'de' in d:
              transl = d['de']
            elif 'en' in d:
              transl = d['en']
            elif 'de' in d:
              transl = d['de']
            else:
              transl = str(d)
        else:
          lform = "(unknown)"
      pr2[(lform, lform, lid, transl)] += p[1]
    pr2 = pr2.most_common()
    pr2 = [[p[1]] + list(p[0]) for p in pr2]
    #print(pr2)
    if False: # Use details/summary elements (old style)
        subtab = tabulate(pr2, headers=[ "Probability", "Guess", "Lemma Form", "Lemma ID", "Translation" ], tablefmt='html')
        s1 = '<details style="display: inline"><summary style="display: inline">'
        s2 = f'</summary> <p style="display: inline">{subtab}</p></details>'
    elif True:
      # Use datalist and input
      s1 = f'<input size="20" style="width: 300px;" type=text list=options-{idx}><!-- ' # value="{pr2[0][1]}"><!--
      s2 = f'--><datalist class="keepDatalistOptions" size="20" id=options-{idx}>'
      for p in pr2:
        s2 += f'<option value="{p[1]}">{p[2]} {p[3]} {p[4]} ({p[0]})'
      s2 += f'</datalist>'
    elif False:
      # Use select/optgroup/option
      js_opts = ''
      # alt behaviour - shows all options:
      size=len(pr2)
      js_opts = f"onfocus='this.size={size*2};' onblur='this.size=1;' onchange='this.size=1; this.blur();'"
      s1 = f'<select style="width: 500px;" type=text {js_opts} value="{pr2[0][1]}"><!-- '
      s2 = f'-->'
      for p in pr2:
        s2 += f'<optgroup label="{p[2]} {p[3]} {p[4]} ({p[0]:0.5f})"><option value="{p[1]}">{p[1]}</optgroup>'

    out.append(
        f"""
           <p>{" ".join(tokens).replace(">", "&extended-gt;").replace("<", s1 + "&lt;").replace("&extended-gt;", "&gt;" + s2)}</p>
        """)
    # optional extra context information
    #out.append("<p>"+(str(t[2]).replace("<", "&lt;").replace(">", "&gt;"))+"</p>")
    #out.append("<p>"+str(t[4])+"</p>")
    #out.append("<p>"+str(t[3])+"</p>")
    out.append("<p>"+str(t[1])+"</p>")
    out.append("<hr>")
  out[-1] = '</form>'
  return "".join(out)
w = widgets.HTML(
    value=get_predictions(slw[:50]),
    placeholder='...Processing...',
    description='Predictions for gaps:')
display(w)




60 suitable sentences


HTML(value='<form>\n      \n           <p>ztp.n sw jtj =f Jmn m <input size="20" style="width: 300px;" type=te…

In [ ]:
# Process predictions with nga_tr_combined
# @markdown Select language to use for translations.
model = nga_tr_combined
prefer_en = True #@param {type:"boolean"}

lemmaform_map = dict()
def to_lemma_id(transliteration):
  if transliteration in lemmaform_map:
    return lemmaform_map[transliteration].most_common(1)[0][0]
  if transliteration in written_form_to_mdc:
    transliteration = written_form_to_mdc[transliteration]
    if transliteration in lemmaform_map:
      return lemmaform_map[transliteration].most_common(1)[0][0]
  return None

for sentence in aecorpus.aes:
  for token in sentence['tokens']:
    if token[2] not in lemmaform_map:
      lemmaform_map[token[2]] = Counter()
    lemmaform_map[token[2]] += Counter({token[3]: 1})

slw = []
cn=0
c=0
gapfile = "snlp-project/preprocessing/final_files/gaps/all_gaps_id_harmonized.txt"
with open(gapfile,"r") as gaps:
  lacunae = re.compile(r'LACUNAE')
  lacunae_word = re.compile(r'<LACUNAE_WORD>')
  for l in gaps.readlines():
    l_ = l.replace("\n", "")
    id,tr = l_.split(" ", 1)
    id = id[1:-1]
    if id not in aes_sentence_dictionary:
      if c < 10:
        print(f"{id} not found")
      cn += 1
      continue
    sd = aes_sentence_dictionary[id]
    tokens = []
    if 'tokens' in sd:
      if '850830' in sd['token_list']:
        continue # Missing item is a name => generally not possible to match with models.
    t = [tr.split(" ")]
    res = lacunae.findall(tr)
    if len(res) == 1 and len(lacunae_word.findall(tr)) == 1:
      slw.append(t)
      c += 1
if cn > 0:
  print(f"{c} suitable sentences; {cn} sentences not found")
else:
  print(f"{c} suitable sentences")
def get_predictions(list_of_inputs):
  predictions = []
  out = [ "" ]
  for idx,t in enumerate(list_of_inputs):
    tokens = t[0]
    transl = None
    #print(tokens)
    pr = predict_words(start_pad() + t[0] + end_pad(),model,lacuna='<LACUNAE_WORD>')
    #print(pr)
    for s in pr:
      if "\n" in s[0]:
        print(t[0])
        print(s[0])
        raise Exception(f"sentence #{idx} {' '.join(tokens)}")
    d = {
      "sentence": " ".join(tokens),
      "predictions": [s[0].replace("\n","") for s in pr]
    }
    if transl is not None:
      d["translation"] = transl
    predictions.append(d)

    words = [p[0] for p in pr]
    s1 = ""
    s2 = ""
    tokens = " ".join(t[0])
    tokens = tokens.replace("<LACUNAE_WORD>", "\u226A"+(" | ".join(words))+"\u226B")
    out.append(
        f"""
           <p>{tokens.replace(">", "&extended-gt;").replace("<", s1 + "&lt;").replace("&extended-gt;", "&gt;" + s2)}</p>
        """)
    if transl is not None:
      out.append(f"<p><i>{transl}</i></p>")
    out.append("<hr>")
  out[-1] = '</form>'
  return ("".join(out), predictions)

html, obj = get_predictions(slw[:50])
w = widgets.HTML(
    value=html,
    placeholder='...Processing...',
    description='Predictions for gaps:')
display(w)

import json
import io

file = "predictions-ngacombined.json"
with open(file, "w") as pf:
  print(json.dumps(obj, indent=2), file=pf)
print(f"Also written predictions to {file}")



IBUBd7TKhu7mKkCKjWrfxxqiU not found
IBcDSKGEkyxdVkVhmy2OjSmFc not found
701 suitable sentences; 104 sentences not found


HTML(value='\n           <p>wD Hm =f Di.t sAH n yty-nTr Mri,y-PtH n ≪=k | =f | =y | pA | =i | tA | kA | nTr | …

Also written predictions to predictions-ngacombined.json


In [ ]:
# Alternative predictions by W2V model.

model=w2v_tr_combined
def get_predictions_w2v(list_of_inputs):
  predictions = []
  out = [ "" ]
  for idx,t in enumerate(list_of_inputs):
    tokens = t[0]
    transl = None
    #print(tokens)
    pr = predict_words(start_pad() + t[0] + end_pad(),model,lacuna='<LACUNAE_WORD>')
    #print(pr)
    for s in pr:
      if "\n" in s[0]:
        print(t[0])
        print(s[0])
        raise Exception(f"sentence #{idx} {' '.join(tokens)}")
    d = {
      "sentence": " ".join(tokens),
      "predictions": [s[0].replace("\n","") for s in pr]
    }
    if transl is not None:
      d["translation"] = transl
    predictions.append(d)

    words = [p[0] for p in pr]
    s1 = ""
    s2 = ""
    tokens = " ".join(t[0])
    tokens = tokens.replace("<LACUNAE_WORD>", "\u226A"+(" | ".join(words))+"\u226B")
    out.append(
        f"""
           <p>{tokens.replace(">", "&extended-gt;").replace("<", s1 + "&lt;").replace("&extended-gt;", "&gt;" + s2)}</p>
        """)
    if transl is not None:
      out.append(f"<p><i>{transl}</i></p>")
    out.append("<hr>")
  out[-1] = '</form>'
  return ("".join(out), predictions)

html, obj = get_predictions_w2v(slw[:50])
w = widgets.HTML(
    value=html,
    placeholder='...Processing...',
    description='Predictions for gaps:')
display(w)

import json
import io

file = "predictions-w2vcombined.json"
with open(file, "w") as pf:
  print(json.dumps(obj, indent=2), file=pf)
print(f"Also written predictions to {file}")



HTML(value='\n           <p>wD Hm =f Di.t sAH n yty-nTr Mri,y-PtH n ≪Wsyr | Hm-nTr | Jy | =k | =f | =i | Wnn-n…

Also written predictions to predictions-w2vcombined.json
